In [ ]:
# ─────────────────────────────────────────────────────────────
# 1. Setup: install deps, load config, init secrets & OpenAI
#    (GitHub Actions-ready: NO Colab upload, NO hardcoded keys)
# ─────────────────────────────────────────────────────────────

import os
import json
import pathlib
from dataclasses import dataclass
from typing import Optional

from openai import OpenAI

# In GitHub Actions, these env vars come from repo secrets:
#   OPENAI_API_KEY
#   SUNO_API_KEY
#   SUNO_BASE_URL   (we will set it as a secret too)
#   YOUTUBE_CLIENT_SECRETS_JSON
#   YOUTUBE_TOKEN_JSON
# Optional later (but not required): YOUTUBE_PRIVACY

def _get_env_required(name: str) -> str:
    value = os.environ.get(name)
    if not value:
        raise ValueError(
            f"Missing required environment variable: {name}.\n"
            "You must add it as a GitHub repository secret and map it in the workflow."
        )
    return value

def _get_env_optional(name: str, default: Optional[str] = None) -> Optional[str]:
    return os.environ.get(name, default)

@dataclass
class Config:
    # Core APIs
    openai_api_key: str
    suno_api_key: str
    suno_base_url: str

    # YouTube auth (stored as JSON strings in GitHub secrets)
    youtube_client_secrets_json: str
    youtube_token_json: str

    # Misc
    youtube_scope: str = "https://www.googleapis.com/auth/youtube.upload"
    youtube_privacy: str = "public"

def load_config() -> Config:
    return Config(
        openai_api_key=_get_env_required("OPENAI_API_KEY"),
        suno_api_key=_get_env_required("SUNO_API_KEY"),
        suno_base_url=_get_env_required("SUNO_BASE_URL"),
        youtube_client_secrets_json=_get_env_required("YOUTUBE_CLIENT_SECRETS_JSON"),
        youtube_token_json=_get_env_required("YOUTUBE_TOKEN_JSON"),
    )

def write_youtube_secret_files(cfg: Config) -> None:
    """
    Turn the JSON strings from env into the files expected by the YouTube client code:
      - client_secrets.json
      - token.json
    """
    client_path = pathlib.Path("client_secrets.json")
    token_path = pathlib.Path("token.json")

    # Validate JSON to fail early if secrets were pasted incorrectly
    try:
        json.loads(cfg.youtube_client_secrets_json)
    except json.JSONDecodeError as e:
        raise ValueError("YOUTUBE_CLIENT_SECRETS_JSON is not valid JSON. Re-paste it into GitHub Secrets.") from e

    try:
        token_data = json.loads(cfg.youtube_token_json)
    except json.JSONDecodeError as e:
        raise ValueError("YOUTUBE_TOKEN_JSON is not valid JSON. Re-paste it into GitHub Secrets.") from e

    # Must have refresh_token for unattended runs
    if "refresh_token" not in token_data:
        raise ValueError(
            "YOUTUBE_TOKEN_JSON is missing 'refresh_token'.\n"
            "You must generate token.json with offline access + youtube.upload scope."
        )

    client_path.write_text(cfg.youtube_client_secrets_json, encoding="utf-8")
    token_path.write_text(cfg.youtube_token_json, encoding="utf-8")

    print(f"✅ Wrote YouTube secrets to {client_path} and {token_path}")

# Load config
config = load_config()

# Initialize OpenAI client
os.environ["OPENAI_API_KEY"] = config.openai_api_key
openai_client = OpenAI(api_key=config.openai_api_key)

# Write YouTube secret files for later blocks
write_youtube_secret_files(config)

print("✅ Config loaded successfully.")
print("   - OPENAI_API_KEY: present")
print("   - SUNO_API_KEY: present")
print(f"   - SUNO_BASE_URL: {config.suno_base_url}")
print("   - YOUTUBE_CLIENT_SECRETS_JSON: present")
print("   - YOUTUBE_TOKEN_JSON: present")
print(f"   - Default YouTube privacy: {config.youtube_privacy}")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 2. Preflight checks: OpenAI, Suno, YouTube, ffmpeg
# ─────────────────────────────────────────────────────────────

import subprocess
import requests
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json

# We reuse: config, openai_client, client_secrets.json, token.json


# ── OpenAI preflight ─────────────────────────────────────────

def preflight_openai(client: OpenAI):
    """
    Simple, cheap call to verify the OpenAI API key works.
    """
    print("🔎 Testing OpenAI connectivity...")
    try:
        models = list(client.models.list())
        print(f"✅ OpenAI OK – {len(models)} models visible.")
    except Exception as e:
        print("❌ OpenAI preflight failed.")
        raise


# ── Suno preflight ───────────────────────────────────────────

def preflight_suno(cfg):
    """
    Light sanity-check for Suno-style API.
    We only check that:
      - A base URL exists
      - We can reach it without network/SSL errors.
    """
    print("🔎 Testing Suno connectivity...")

    base_url = cfg.suno_base_url
    if not base_url:
        raise ValueError(
            "Suno base URL is not set.\n"
            "Set SUNO_BASE_URL in Colab (Block 1) or as a GitHub secret later.\n"
            "Example: https://api.sunoapi.org/api/v1"
        )

    base_url = base_url.rstrip("/")
    headers = {"Authorization": f"Bearer {cfg.suno_api_key}"}

    try:
        # Very cheap connectivity check:
        resp = requests.get(base_url, headers=headers, timeout=10)
        print(f"✅ Suno connectivity OK – HTTP {resp.status_code} from {base_url}")
    except Exception as e:
        print("❌ Suno preflight failed.")
        raise


# ── YouTube auth & preflight ─────────────────────────────────

SCOPE = config.youtube_scope  # same as we defined in Config


def get_authenticated_service():
    """Create authenticated YouTube client with scope validation."""
    try:
        # Load token data from the file we wrote in Block 1
        with open("token.json") as f:
            token_data = json.load(f)

        # Verify the token has our required scope
        if "scopes" not in token_data or SCOPE not in token_data["scopes"]:
            raise ValueError(f"Token missing required scope: {SCOPE}")

        creds = Credentials.from_authorized_user_info(token_data, [SCOPE])

        # Refresh token if needed
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
                # Update token file with refreshed credentials
                with open("token.json", "w") as f:
                    json.dump(json.loads(creds.to_json()), f)
                print("🔄 YouTube token refreshed.")
            except Exception as refresh_error:
                print(f"⚠️ Token refresh failed: {refresh_error}")
                if not creds.token:
                    raise

        return build("youtube", "v3", credentials=creds)
    except HttpError as e:
        print(f"❌ YouTube API error: {e}")
        raise
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        raise


def preflight_youtube():
    """
    Confirm that we can load and refresh YouTube credentials.
    We DON'T call any specific API endpoint that requires extra scopes,
    because our token only guarantees youtube.upload.
    """
    print("🔎 Testing YouTube authentication...")
    yt = get_authenticated_service()
    # If we got here without exception, credentials & scope are fine for uploads
    print("✅ YouTube credentials OK – ready for uploads.")
    return yt


# ── ffmpeg preflight ─────────────────────────────────────────

def preflight_ffmpeg():
    """
    Ensure ffmpeg is installed and callable.
    Colab usually has ffmpeg preinstalled, but we confirm.
    """
    print("🔎 Testing ffmpeg availability...")
    try:
        result = subprocess.run(
            ["ffmpeg", "-version"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
        )
        if result.returncode == 0:
            first_line = result.stdout.splitlines()[0] if result.stdout else "ffmpeg available"
            print(f"✅ ffmpeg OK – {first_line}")
        else:
            print("❌ ffmpeg returned a non-zero exit code.")
            print(result.stderr[:500])
            raise RuntimeError("ffmpeg not working properly.")
    except FileNotFoundError:
        print("❌ ffmpeg not found on PATH.")
        raise
    except Exception as e:
        print("❌ ffmpeg preflight failed.")
        raise


# ── Run all preflights ───────────────────────────────────────

print("🚀 Running preflight checks...\n")

preflight_openai(openai_client)
preflight_suno(config)
youtube_client = preflight_youtube()
preflight_ffmpeg()

print("\n✅ All preflight checks passed. Safe to proceed with the rest of the pipeline.")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 3. Topic / Persona / Tone / Format selection + run metadata
#    - Multi-layer topic engine (families + seasonal + trending + rotation)
#    - Content diversity engine via "formats" (maps to persona + tone)
#    - Logs WHY a subject was chosen (seed vs trending vs mixed)
#    - UPDATED trending: multi-day pool + random sampling + "only if relevant"
#    - NEW: Soft title-length guidance for Suno (<= 80 chars) via GPT rewrite (no trimming)
# ─────────────────────────────────────────────────────────────

import random
import uuid
from datetime import datetime, timedelta, timezone
import pathlib
import json
import re
import requests

# We reuse:
# - openai_client (from Block 1)

BASE_OUT_DIR = pathlib.Path("outputs")
BASE_OUT_DIR.mkdir(parents=True, exist_ok=True)

RECENT_SUBJECT_WINDOW = 50
SEASONAL_FAMILY_BIAS_PROB = 0.7
TOPIC_MODEL = "gpt-4.1-mini"

# ─────────────────────────────────────────────────────────────
# Shorts-friendly length targets (for Block 4 lyric generation)
# ─────────────────────────────────────────────────────────────
SHORTS_TARGET_SECONDS = 120       # ideal target ~2:00
SHORTS_SOFT_MAX_SECONDS = 175     # practical "try to stay under" (~2:55)

# ─────────────────────────────────────────────────────────────
# Suno title constraint (soft guidance; NO hard trimming)
# Suno error: "music title cannot exceed 80 characters"
# We'll guide GPT to stay under this, and do a gentle rewrite if it doesn't.
# ─────────────────────────────────────────────────────────────
SUNO_TITLE_MAX_CHARS = 80
SUNO_TITLE_SOFT_MAX_CHARS = 70   # aim below hard cap to reduce edge-case failures

# ─────────────────────────────────────────────────────────────
# Subject families with more general-knowledge, view-friendly seeds
# (Your preferred list; songs still explain deeply downstream)
# ─────────────────────────────────────────────────────────────

SUBJECT_FAMILIES = {
    "science_physics_space": [
        "black holes",
        "gravity in everyday life",
        "relativity explained simply",
        "why the sky is blue",
        "how rockets escape Earth",
        "time dilation in simple words",
        "what dark matter might be",
        "how GPS knows your location",
        "why orbits don't fall down",
    ],
    "science_energy_earth": [
        "how solar panels work",
        "how batteries store energy",
        "the greenhouse effect",
        "nuclear fusion vs fission",
        "how wind turbines generate power",
        "the water cycle",
        "climate change basics",
        "why seasons exist",
        "how electricity gets to your home",
    ],
    "science_bio_body": [
        "evolution by natural selection",
        "photosynthesis",
        "how vaccines work",
        "the human immune system basics",
        "how neurons send signals",
        "sleep cycles and circadian rhythm",
        "how digestion works",
        "how your heart pumps blood",
    ],
    "finance_and_economics": [
        "inflation",
        "compound interest",
        "tax brackets",
        "budgeting for beginners",
        "what a stock is",
        "what ETFs are",
        "index funds vs stock picking",
        "how mortgages work",
        "good debt vs bad debt",
        "credit scores explained",
        "what GDP actually measures",
        "how central banks influence interest rates",
        "how government debt works",
    ],
    "history_and_society": [
        "the fall of the Roman Empire",
        "World War 2 in Europe (very high-level)",
        "the French Revolution",
        "the industrial revolution",
        "the invention of the internet",
        "the printing press and its impact",
        "the Cold War in simple terms",
        "the rise of social media",
        "the history of money",
        "how democracies develop over time",
    ],
    "psychology_and_life_skills": [
        "procrastination",
        "habits and habit loops",
        "impostor syndrome",
        "growth mindset",
        "how memory works",
        "decision fatigue",
        "dopamine and motivation",
        "introvert vs extrovert myths",
        "cognitive biases in everyday life",
        "how to build discipline",
        "why we self-sabotage",
    ],
    "tech_and_digital_life": [
        "how the internet works",
        "what algorithms are",
        "how recommendation systems work",
        "what a blockchain is",
        "password security basics",
        "cloud computing explained simply",
        "artificial intelligence vs machine learning",
        "how encryption keeps data safe",
        "how social media hooks your brain",
    ],
    "internet_and_creator_economy": [
        "how the YouTube algorithm picks your next video",
        "what watch time actually means for creators",
        "why Shorts go viral but long videos don't",
        "how TikTok decides what you see",
        "what engagement really is on social media",
        "how influencers actually make money",
        "what brand deals and sponsorships really are",
        "why some channels blow up overnight",
        "what a content niche is and why it matters",
        "how click-through rate affects your views",
    ],
    "gaming_and_virtual_worlds": [
        "how Minecraft redstone is basically real circuits",
        "why Minecraft is perfect for learning logic",
        "what game XP teaches you about habits",
        "how matchmaking and skill rating work in online games",
        "loot boxes vs gambling in games",
        "why Roblox is a whole economy not just a game",
        "game currencies vs real money",
        "how anti-cheat systems try to catch hackers",
        "ping, lag and servers explained",
        "what game design tricks keep you playing",
    ],
    "ai_and_future_of_work": [
        "how AI chatbots actually work in simple terms",
        "why AI hallucinates wrong answers",
        "what training data really is",
        "how image generators turn text into art",
        "will AI take all jobs or just some tasks",
        "what automation really means for your future work",
        "how AI filters your social media feed",
        "deepfakes explained in simple words",
        "how to spot AI-generated content online",
        "why AI still needs human supervision",
    ],
    "modern_money_and_side_hustles": [
        "how affiliate marketing works in 60 seconds",
        "dropshipping explained without the fake guru BS",
        "what passive income really is and isn't",
        "how taxes work when you make money online",
        "why get rich quick is almost always a scam",
        "how to budget your first paycheck",
        "difference between salary hourly and freelance work",
        "why investing beats saving over time",
        "side hustles that are actually skills not schemes",
        "how creators sell digital products",
    ],
    "digital_safety_and_privacy": [
        "why password managers exist",
        "how hackers guess your passwords",
        "what phishing emails look like",
        "why public wifi isn't really safe",
        "what a VPN actually does",
        "how end to end encryption keeps chats private",
        "what cookies and tracking really are",
        "how apps sell your data in plain language",
        "what two factor authentication protects you from",
        "how to spot a scam website",
    ],
    "world_systems_and_geopolitics": [
        "how elections work in a democracy",
        "what the United Nations actually does",
        "what NATO is and why it was created",
        "how sanctions between countries work",
        "why interest rate decisions affect the entire world",
        "what a trade war actually is",
        "how oil prices affect your groceries",
        "what the World Bank and IMF do",
        "why global supply chains are so fragile",
        "how borders between countries are decided",
    ],
    "internet_culture_and_psychology": [
        "why memes spread like viruses",
        "what doomscrolling does to your brain",
        "why you can't stop checking notifications",
        "FOMO explained in one minute",
        "why comment sections are so toxic",
        "paradox of choice and too many options",
        "why outrage content gets more views",
        "what parasocial relationships really are",
        "why you feel like everyone else is doing better",
        "how echo chambers can change your views",
    ],
    "learning_and_school_survival": [
        "how memory actually works and how to hack it",
        "spaced repetition explained with a song",
        "why cramming the night before doesn't work",
        "how to take notes so you actually remember",
        "what focus really is and why you lose it",
        "how to beat procrastination on homework",
        "difference between studying hard and studying smart",
        "how to use the Pomodoro technique for school",
        "why multitasking kills your grades",
        "what growth mindset means for exams",
    ],
    "everyday_science_and_tech": [
        "how your phone battery actually works",
        "why your phone overheats when gaming",
        "how noise cancelling headphones trick your ears",
        "why airplanes stay up in the sky",
        "why your shower water changes temperature suddenly",
        "how wifi sends data through the air",
        "why microwaves heat food from inside",
        "why some ice cubes are cloudy",
        "how credit card tap to pay works",
        "how GPS knows where you are",
    ],
    "myths_and_misconceptions": [
        "no you don't use only ten percent of your brain",
        "is sugar really as addictive as drugs",
        "do phone chargers drain power when not in use",
        "does detox actually mean anything",
        "is multitasking a real skill",
        "will cracking your knuckles give you arthritis",
        "does eating late make you fat",
        "is eight hours of sleep the only correct amount",
        "are left brained vs right brained people real",
        "do you really need ten thousand steps a day",
    ],
    "sports_and_performance_science": [
        "what expected goals xG means in football",
        "how player ratings are calculated in games like FIFA",
        "why some players choke under pressure",
        "how sprint speed and acceleration really work",
        "why stamina wins more matches than talent",
        "what sports analytics teams do behind the scenes",
        "how reaction time can be trained",
        "why home advantage exists in sports",
        "how strategy changes in the final minutes of a game",
        "how injuries actually happen in sports",
    ],
    "climate_and_future_cities": [
        "how carbon footprints are actually calculated",
        "what net zero really means",
        "are electric cars really better for the planet",
        "how smart cities use data to run everything",
        "what a carbon tax is",
        "how heat pumps warm your house efficiently",
        "why cities are getting hotter over time",
        "how public transport systems are designed",
        "how renewable energy fits into the grid",
        "what climate tipping points are",
    ],
}

# ─────────────────────────────────────────────────────────────
# Format engine
# ─────────────────────────────────────────────────────────────

FORMAT_DEFS = {
    "explain_like_im_5": {
        "label": "Explain Like I'm 5",
        "persona": "lofi chillhop, soft and cozy",
        "tone": "wholesome and uplifting",
        "background_style": "simple cute illustration, soft pastel colors, rounded shapes, cozy classroom vibe",
    },
    "epic_breakdown": {
        "label": "Epic Breakdown",
        "persona": "epic cinematic soundtrack, big drums",
        "tone": "dramatic and intense",
        "background_style": "cinematic wide shot, dramatic lighting, epic atmosphere",
    },
    "science_pop_song": {
        "label": "Science Pop Song",
        "persona": "upbeat pop-punk, high energy guitars",
        "tone": "motivational and energetic",
        "background_style": "bright bold colors, energetic composition, playful science icons",
    },
    "rap_summary": {
        "label": "Rap Summary",
        "persona": "dark trap, heavy 808s, moody",
        "tone": "sarcastic but friendly",
        "background_style": "urban neon style, bold contrast, dynamic graffiti-like shapes",
    },
    "history_musical": {
        "label": "History Musical",
        "persona": "Disney-style musical, orchestral pop",
        "tone": "thoughtful and reflective",
        "background_style": "painterly historical scenes, warm lighting, storytelling composition",
    },
    "mindset_diary": {
        "label": "Mindset Diary",
        "persona": "indie folk, acoustic and warm",
        "tone": "calm and reassuring",
        "background_style": "soft vignette, cozy room, journal-style illustration",
    },
    "techwave": {
        "label": "Techwave",
        "persona": "synthwave, nostalgic 80s electronic",
        "tone": "mysterious and slightly spooky",
        "background_style": "neon grids, abstract tech shapes, glowing circuitry",
    },
}

FAMILY_FORMAT_MAP = {
    "science_physics_space": ["explain_like_im_5", "epic_breakdown", "science_pop_song", "techwave"],
    "science_energy_earth": ["explain_like_im_5", "science_pop_song", "epic_breakdown"],
    "science_bio_body": ["explain_like_im_5", "science_pop_song", "mindset_diary"],
    "finance_and_economics": ["rap_summary", "explain_like_im_5", "mindset_diary"],
    "history_and_society": ["history_musical", "epic_breakdown", "explain_like_im_5"],
    "psychology_and_life_skills": ["mindset_diary", "explain_like_im_5", "rap_summary"],
    "tech_and_digital_life": ["techwave", "science_pop_song", "explain_like_im_5"],

    "internet_and_creator_economy": ["rap_summary", "techwave", "explain_like_im_5"],
    "gaming_and_virtual_worlds": ["techwave", "science_pop_song", "rap_summary"],
    "ai_and_future_of_work": ["techwave", "science_pop_song", "epic_breakdown"],
    "modern_money_and_side_hustles": ["rap_summary", "explain_like_im_5", "mindset_diary"],
    "digital_safety_and_privacy": ["explain_like_im_5", "epic_breakdown", "techwave"],
    "world_systems_and_geopolitics": ["history_musical", "epic_breakdown", "explain_like_im_5"],
    "internet_culture_and_psychology": ["rap_summary", "mindset_diary", "techwave"],
    "learning_and_school_survival": ["explain_like_im_5", "mindset_diary", "science_pop_song"],
    "everyday_science_and_tech": ["explain_like_im_5", "science_pop_song", "techwave"],
    "myths_and_misconceptions": ["explain_like_im_5", "rap_summary", "science_pop_song"],
    "sports_and_performance_science": ["science_pop_song", "epic_breakdown", "mindset_diary"],
    "climate_and_future_cities": ["science_pop_song", "epic_breakdown", "explain_like_im_5"],
}

# ─────────────────────────────────────────────────────────────
# Seasonal profiles
# ─────────────────────────────────────────────────────────────

def get_seasonal_profile(now: datetime) -> dict:
    m = now.month
    if m == 1:
        return {
            "label": "New Year: money, habits & fresh start",
            "preferred_families": [
                "finance_and_economics",
                "modern_money_and_side_hustles",
                "psychology_and_life_skills",
                "learning_and_school_survival",
            ],
            "notes": "New year goals, money clean-up, habit building, self-improvement."
        }
    elif m == 2:
        return {
            "label": "Relationships, mind & digital life",
            "preferred_families": [
                "psychology_and_life_skills",
                "science_bio_body",
                "internet_culture_and_psychology",
            ],
            "notes": "Emotions, social life, online relationships, how our body/brain works."
        }
    elif m in (3, 4):
        return {
            "label": "Spring: energy, nature & environment",
            "preferred_families": [
                "science_energy_earth",
                "science_bio_body",
                "climate_and_future_cities",
                "everyday_science_and_tech",
            ],
            "notes": "Energy, climate, biology waking up, environment and cities."
        }
    elif m in (5, 6):
        return {
            "label": "End of school year: history & big ideas",
            "preferred_families": [
                "history_and_society",
                "science_physics_space",
                "world_systems_and_geopolitics",
            ],
            "notes": "History recaps, big systems, how the world fits together."
        }
    elif m in (7, 8):
        return {
            "label": "Summer curiosity: games, world & tech",
            "preferred_families": [
                "science_energy_earth",
                "tech_and_digital_life",
                "gaming_and_virtual_worlds",
                "internet_and_creator_economy",
            ],
            "notes": "Weird nature, gaming, creators, tech deep-dives while people have free time."
        }
    elif m == 9:
        return {
            "label": "Back to school: basics & survival",
            "preferred_families": [
                "science_physics_space",
                "finance_and_economics",
                "psychology_and_life_skills",
                "learning_and_school_survival",
                "everyday_science_and_tech",
            ],
            "notes": "Core concepts for school and life, study hacks and practical basics."
        }
    elif m in (10,):
        return {
            "label": "Spooky science, mind & myths",
            "preferred_families": [
                "science_physics_space",
                "psychology_and_life_skills",
                "myths_and_misconceptions",
                "internet_culture_and_psychology",
            ],
            "notes": "Slightly spooky science, mind, and busting viral myths."
        }
    elif m in (11,):
        return {
            "label": "Money, systems & the world",
            "preferred_families": [
                "finance_and_economics",
                "modern_money_and_side_hustles",
                "history_and_society",
                "world_systems_and_geopolitics",
                "ai_and_future_of_work",
            ],
            "notes": "Economy, world systems, future of work before year-end."
        }
    else:  # December
        return {
            "label": "Year-in-review, world & future",
            "preferred_families": [
                "history_and_society",
                "tech_and_digital_life",
                "science_physics_space",
                "world_systems_and_geopolitics",
                "internet_and_creator_economy",
                "ai_and_future_of_work",
            ],
            "notes": "Inventions, big events, creators, AI and where the world is heading."
        }

# ─────────────────────────────────────────────────────────────
# History: recent subjects
# ─────────────────────────────────────────────────────────────

def load_recent_subjects(base_dir: pathlib.Path, window: int = RECENT_SUBJECT_WINDOW) -> list[str]:
    if not base_dir.exists():
        return []
    meta_files = sorted(
        base_dir.glob("run_*/run_meta.json"),
        key=lambda p: p.stat().st_mtime,
        reverse=True,
    )
    recent_subjects = []
    for mf in meta_files[:window]:
        try:
            data = json.loads(mf.read_text(encoding="utf-8"))
            subj = data.get("subject")
            if subj:
                recent_subjects.append(subj)
        except Exception:
            continue
    return recent_subjects

# ─────────────────────────────────────────────────────────────
# Trending pulse (UPDATED)
# ─────────────────────────────────────────────────────────────

TREND_DAYS_LOOKBACK = 10           # scan last 10 days
TREND_POOL_MAX_ITEMS = 30          # final deduped pool size
TREND_GPT_SAMPLE_K = 7             # how many keywords we pass into GPT each run
TREND_MIN_USE_PROB = 0.55          # even if we have trends, only include them ~55% of the time

def fetch_trending_keywords(
    max_items: int = TREND_POOL_MAX_ITEMS,
    days_lookback: int = TREND_DAYS_LOOKBACK,
) -> dict:
    """
    Returns:
      {
        "pool": [kw1, kw2, ...],        # deduped pool (size <= max_items)
        "sample_for_gpt": [...],        # random subset for this run
        "source": "wikipedia_pageviews",
        "days_used": [date_strs...],
        "errors": [ ... ],
      }
    """
    def _fetch_for_date(d):
        url = (
            "https://wikimedia.org/api/rest_v1/metrics/pageviews/top/"
            f"en.wikipedia/all-access/{d.year}/{d.month:02d}/{d.day:02d}"
        )
        headers = {
            "User-Agent": "suno-shorts-bot/1.0 (contact: your-email@example.com)"
        }
        r = requests.get(url, headers=headers, timeout=12)
        r.raise_for_status()
        data = r.json()
        items = data.get("items", [])
        if not items:
            return []
        articles = items[0].get("articles", [])
        out = []
        for art in articles:
            title = art.get("article", "")
            if not title:
                continue
            if title in ("Main_Page", "Special:Search"):
                continue
            if ":" in title:
                continue

            clean = title.replace("_", " ").strip()

            if len(clean) < 3 or len(clean) > 45:
                continue
            if re.fullmatch(r"\d+", clean):
                continue
            if re.search(r"[<>@#%{}|\\^~\[\]]", clean):
                continue

            out.append(clean)

        return out

    today = datetime.now(timezone.utc).date()

    errors = []
    used_days = []
    collected = []

    for delta in range(2, 2 + days_lookback):
        d = today - timedelta(days=delta)
        try:
            kws = _fetch_for_date(d)
            if kws:
                used_days.append(str(d))
                collected.extend(kws)
        except Exception as e:
            errors.append(f"{d}: {e}")

    seen = set()
    deduped = []
    for kw in collected:
        k = kw.strip()
        kl = k.lower()
        if kl in seen:
            continue
        seen.add(kl)
        deduped.append(k)

    soft_downweight_terms = {
        "ai", "artificial intelligence", "chatgpt", "openai",
        "machine learning", "deep learning", "llm", "gpt"
    }

    aiish = []
    non_aiish = []
    for kw in deduped:
        low = kw.lower()
        if any(t in low for t in soft_downweight_terms):
            aiish.append(kw)
        else:
            non_aiish.append(kw)

    final_pool = []
    final_pool.extend(non_aiish[:max_items])
    remaining = max(0, max_items - len(final_pool))
    if remaining > 0:
        final_pool.extend(aiish[: min(3, remaining)])

    if len(final_pool) < 10:
        remaining = max(0, max_items - len(final_pool))
        final_pool.extend(aiish[:remaining])

    if final_pool:
        k = min(TREND_GPT_SAMPLE_K, len(final_pool))
        sample_for_gpt = random.sample(final_pool, k=k)
    else:
        sample_for_gpt = []

    return {
        "pool": final_pool[:max_items],
        "sample_for_gpt": sample_for_gpt,
        "source": "wikipedia_pageviews",
        "days_used": used_days,
        "errors": errors,
    }

# ─────────────────────────────────────────────────────────────
# GPT topic generation (UPDATED: "only if relevant" + title length guidance)
# ─────────────────────────────────────────────────────────────

def gpt_generate_topic_from_context(
    family_name: str,
    base_ideas: list[str],
    recent_subjects: list[str],
    seasonal_profile: dict,
    trending_keywords: list[str],
    language: str = "English",
) -> str:
    recent_snippet = ""
    if recent_subjects:
        recent_snippet = "\nRecently used topics (avoid repeating these exactly):\n- " + "\n- ".join(
            recent_subjects[:20]
        )

    seasonal_label = seasonal_profile.get("label", "General")
    seasonal_fams = seasonal_profile.get("preferred_families", [])
    seasonal_notes = seasonal_profile.get("notes", "")

    trending_snippet = ""
    if trending_keywords:
        trending_snippet = (
            "\nTrending keywords (random sample). Use them ONLY if genuinely relevant to this family.\n"
            "If none are relevant, IGNORE them completely.\n"
            "- " + "\n- ".join(trending_keywords)
        )

    prompt = (
        f"You are helping design topics for educational YouTube Shorts songs in {language}.\n\n"
        f"Subject family: {family_name}\n"
        "Base seed ideas (this is your main source):\n"
        + "\n".join(f"- {idea}" for idea in base_ideas)
        + "\n"
        + recent_snippet
        + "\n\nSeasonal context right now:\n"
        f"- Theme label: {seasonal_label}\n"
        f"- Seasonal notes: {seasonal_notes}\n"
        f"- Seasonal preferred families: {', '.join(seasonal_fams)}\n"
        + (trending_snippet if trending_keywords else "\n(No trending keywords available for this run.)")
        + "\n\nRules for using trends:\n"
        "- First pick a strong topic based on the BASE SEEDS.\n"
        "- Then check the trending keywords: if ONE is clearly relevant, you may gently adapt the topic to connect.\n"
        "- Do NOT pivot the whole topic to something unrelated.\n"
        "- If nothing is relevant, ignore trends.\n\n"
        "Your task:\n"
        "- Output ONE specific, engaging topic title for an educational song.\n"
        "- Short (3–8 words), clear, beginner-friendly.\n"
        "- Safe for a general audience.\n"
        "- Politics/geopolitics allowed ONLY neutrally; no endorsements.\n"
        "- Do NOT repeat any recently used topic exactly.\n"
        f"- IMPORTANT: This will be used as a music TITLE. Aim <= {SUNO_TITLE_SOFT_MAX_CHARS} characters.\n"
        "- No quotes, emojis, or explanations.\n"
        "- Return ONLY the topic line.\n"
    )

    resp = openai_client.chat.completions.create(
        model=TOPIC_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=64,
    )

    topic = resp.choices[0].message.content.strip()

    if topic.startswith('"') and topic.endswith('"'):
        topic = topic[1:-1].strip()
    if topic.startswith("'") and topic.endswith("'"):
        topic = topic[1:-1].strip()

    if len(topic.split()) > 10:
        topic = " ".join(topic.split()[:10])

    return topic

def gpt_rewrite_short_title(
    topic: str,
    family_name: str,
    base_ideas: list[str],
    trending_keywords: list[str],
    language: str = "English",
) -> str:
    """
    If GPT returns a title that's too long for Suno, ask GPT to rewrite it shorter.
    This is NOT trimming; it's regeneration with constraints.
    """
    seeds = "\n".join(f"- {i}" for i in base_ideas[:6])
    trends = "\n".join(f"- {k}" for k in trending_keywords[:7]) if trending_keywords else "(none)"

    prompt = (
        f"Rewrite this music title to be SHORTER for YouTube + music platforms.\n\n"
        f"Original title:\n{topic}\n\n"
        f"Context:\n- Family: {family_name}\n- Seed ideas:\n{seeds}\n- Trending keywords (optional):\n{trends}\n\n"
        "Rules:\n"
        "- Keep the SAME meaning, just shorter and cleaner.\n"
        "- Stay beginner-friendly, but still specific.\n"
        "- No hype, no emojis, no quotes.\n"
        f"- HARD REQUIREMENT: <= {SUNO_TITLE_MAX_CHARS} characters.\n"
        f"- Aim <= {SUNO_TITLE_SOFT_MAX_CHARS} characters if possible.\n"
        "- Output ONLY the rewritten title line.\n"
    )

    resp = openai_client.chat.completions.create(
        model=TOPIC_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
        max_tokens=64,
    )
    out = resp.choices[0].message.content.strip()

    if out.startswith('"') and out.endswith('"'):
        out = out[1:-1].strip()
    if out.startswith("'") and out.endswith("'"):
        out = out[1:-1].strip()

    if len(out.split()) > 10:
        out = " ".join(out.split()[:10])

    return out

# ─────────────────────────────────────────────────────────────
# Topic origin classifier (seed vs trending vs mixed)
# ─────────────────────────────────────────────────────────────

def classify_topic_origin(topic: str, base_ideas: list[str], trending_keywords: list[str]) -> tuple[str, dict]:
    tl = topic.lower()

    matched_trending = []
    for kw in trending_keywords:
        kl = kw.lower()
        if kl in tl or tl in kl:
            matched_trending.append(kw)

    matched_seeds = []
    for idea in base_ideas:
        il = idea.lower()
        if il in tl or tl in il:
            matched_seeds.append(idea)

    if matched_trending and not matched_seeds:
        origin = "trending_inspired"
    elif matched_seeds and not matched_trending:
        origin = "seed_based"
    elif matched_seeds and matched_trending:
        origin = "mixed_seed_and_trending"
    else:
        origin = "mixed_or_derived"

    details = {
        "matched_trending_keywords": matched_trending,
        "matched_seed_ideas": matched_seeds,
    }
    return origin, details

# ─────────────────────────────────────────────────────────────
# Format picker
# ─────────────────────────────────────────────────────────────

def pick_format_for_family(family_name: str):
    formats = FAMILY_FORMAT_MAP.get(family_name) or list(FORMAT_DEFS.keys())
    fmt_id = random.choice(formats)
    fmt_cfg = FORMAT_DEFS[fmt_id]
    return fmt_id, fmt_cfg

# ─────────────────────────────────────────────────────────────
# Run directory helper
# ─────────────────────────────────────────────────────────────

def create_new_run_dir(base_dir: pathlib.Path = BASE_OUT_DIR) -> pathlib.Path:
    timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%d_%H-%M-%S")
    short_id = uuid.uuid4().hex[:8]
    run_name = f"run_{timestamp}_{short_id}"
    run_dir = base_dir / run_name
    run_dir.mkdir(parents=True, exist_ok=True)
    (run_dir / "audio").mkdir(exist_ok=True)
    (run_dir / "video").mkdir(exist_ok=True)
    (run_dir / "images").mkdir(exist_ok=True)
    (run_dir / "logs").mkdir(exist_ok=True)
    return run_dir

# ─────────────────────────────────────────────────────────────
# Orchestrate
# ─────────────────────────────────────────────────────────────

def pick_subject_and_format(base_dir: pathlib.Path):
    now = datetime.now(timezone.utc)
    seasonal_profile = get_seasonal_profile(now)
    recent_subjects = load_recent_subjects(base_dir, RECENT_SUBJECT_WINDOW)

    # 1) Choose family (seasonal bias)
    all_fams = list(SUBJECT_FAMILIES.keys())
    seasonal_fams = [
        fam for fam in seasonal_profile.get("preferred_families", []) if fam in SUBJECT_FAMILIES
    ]
    if seasonal_fams and random.random() < SEASONAL_FAMILY_BIAS_PROB:
        family_name = random.choice(seasonal_fams)
        seasonal_choice = True
    else:
        family_name = random.choice(all_fams)
        seasonal_choice = False

    base_list = SUBJECT_FAMILIES[family_name]
    num_seeds = min(5, len(base_list))
    base_ideas = random.sample(base_list, k=num_seeds)

    # 2) Trending: build pool from last N days, then provide random sample for GPT sometimes
    trend_pack = fetch_trending_keywords(max_items=TREND_POOL_MAX_ITEMS, days_lookback=TREND_DAYS_LOOKBACK)
    trend_pool = trend_pack["pool"]
    if trend_pool and (random.random() < TREND_MIN_USE_PROB):
        trending_for_gpt = trend_pack["sample_for_gpt"]
        trending_used = True
    else:
        trending_for_gpt = []
        trending_used = False

    try:
        topic = gpt_generate_topic_from_context(
            family_name=family_name,
            base_ideas=base_ideas,
            recent_subjects=recent_subjects,
            seasonal_profile=seasonal_profile,
            trending_keywords=trending_for_gpt,
            language="English",
        )

        # If title too long for Suno, do ONE rewrite attempt (no trimming).
        if len(topic) > SUNO_TITLE_MAX_CHARS:
            print(f"⚠️ Topic title too long for Suno ({len(topic)} chars). Rewriting shorter...")
            topic2 = gpt_rewrite_short_title(
                topic=topic,
                family_name=family_name,
                base_ideas=base_ideas,
                trending_keywords=trending_for_gpt,
                language="English",
            )
            print(f"   → Rewritten: {topic2} ({len(topic2)} chars)")
            topic = topic2

        if topic in recent_subjects:
            print("⚠️ GPT topic was in recent subjects, falling back to a base idea.")
            topic = random.choice(base_ideas)

        topic_origin, origin_details = classify_topic_origin(topic, base_ideas, trending_for_gpt)

        print(
            f"🎯 GPT-chosen topic: {topic} "
            f"(family: {family_name}, seasonal_bias={seasonal_choice}, trending_in_prompt={trending_used})"
        )

    except Exception as e:
        print("⚠️ GPT topic generation failed, falling back to random base idea:", e)
        topic = random.choice(base_list)
        topic_origin = "seed_based_fallback"
        origin_details = {"matched_trending_keywords": [], "matched_seed_ideas": [topic]}
        print(f"🎯 Fallback topic: {topic} (family: {family_name})")

    # 3) Format
    format_id, fmt_cfg = pick_format_for_family(family_name)
    persona = fmt_cfg["persona"]
    tone = fmt_cfg["tone"]
    format_label = fmt_cfg["label"]
    background_style_hint = fmt_cfg["background_style"]

    # Explanation text
    seasonal_mode = "seasonal family" if seasonal_choice else "non-seasonal random family"
    trend_desc = (
        "no trending pool available"
        if not trend_pool
        else f"trend pool size={len(trend_pool)} (lookback={TREND_DAYS_LOOKBACK}d)"
    )
    trend_prompt_desc = (
        "trending NOT used in GPT prompt"
        if not trending_for_gpt
        else f"trending sample used in prompt={trending_for_gpt}"
    )

    origin_text = (
        f"Family chosen via {seasonal_mode}. "
        f"Base seeds sampled: {base_ideas}. "
        f"{trend_desc}. {trend_prompt_desc}. "
        f"Topic classified as '{topic_origin}'."
    )

    return {
        "subject": topic,
        "family_name": family_name,
        "format_id": format_id,
        "format_label": format_label,
        "persona": persona,
        "tone": tone,
        "background_style_hint": background_style_hint,
        "season_label": seasonal_profile.get("label", "General"),
        "seasonal_choice": seasonal_choice,
        # Trending debug
        "trend_source": trend_pack.get("source"),
        "trend_days_used": trend_pack.get("days_used", []),
        "trend_errors": trend_pack.get("errors", []),
        "trending_pool": trend_pool[:15],
        "trending_used_in_prompt": trending_used,
        "trending_prompt_sample": trending_for_gpt,
        # Topic selection info
        "base_ideas_used": base_ideas,
        "topic_origin": topic_origin,
        "topic_origin_details": origin_details,
        "topic_selection_explanation": origin_text,
        # Title-length debug
        "suno_title_max_chars": SUNO_TITLE_MAX_CHARS,
        "suno_title_soft_max_chars": SUNO_TITLE_SOFT_MAX_CHARS,
        "subject_char_count": len(topic),
    }

# ─────────────────────────────────────────────────────────────
# Create run + pick theme
# ─────────────────────────────────────────────────────────────

RUN_DIR = create_new_run_dir()
pick_result = pick_subject_and_format(BASE_OUT_DIR)

subject = pick_result["subject"]
persona = pick_result["persona"]
tone = pick_result["tone"]

run_meta = {
    "run_dir": str(RUN_DIR),
    "created_utc": datetime.now(timezone.utc).isoformat(),
    "subject": subject,
    "topic_family": pick_result["family_name"],
    "format_id": pick_result["format_id"],
    "format_label": pick_result["format_label"],
    "persona": persona,
    "tone": tone,
    "season_label": pick_result["season_label"],
    "seasonal_choice": pick_result["seasonal_choice"],
    "background_style_hint": pick_result["background_style_hint"],

    # Topic engine logs
    "base_ideas_used": pick_result["base_ideas_used"],
    "topic_origin": pick_result["topic_origin"],
    "topic_origin_details": pick_result["topic_origin_details"],
    "topic_selection_explanation": pick_result["topic_selection_explanation"],

    # Trending logs
    "trend_source": pick_result["trend_source"],
    "trend_days_used": pick_result["trend_days_used"],
    "trend_errors": pick_result["trend_errors"],
    "trending_pool_preview": pick_result["trending_pool"],
    "trending_used_in_prompt": pick_result["trending_used_in_prompt"],
    "trending_prompt_sample": pick_result["trending_prompt_sample"],

    # Title constraints for Suno
    "suno_title_max_chars": pick_result.get("suno_title_max_chars", SUNO_TITLE_MAX_CHARS),
    "suno_title_soft_max_chars": pick_result.get("suno_title_soft_max_chars", SUNO_TITLE_SOFT_MAX_CHARS),
    "subject_char_count": pick_result.get("subject_char_count", len(subject)),

    # Length targets for Block 4 lyric generation
    "shorts_target_seconds": SHORTS_TARGET_SECONDS,
    "shorts_soft_max_seconds": SHORTS_SOFT_MAX_SECONDS,
}

meta_path = RUN_DIR / "run_meta.json"
meta_path.write_text(json.dumps(run_meta, indent=2, ensure_ascii=False))

print("🎬 New run initialized:")
print("  Run dir      :", RUN_DIR)
print("  Subject      :", subject, f"({len(subject)} chars)")
print("  Topic family :", pick_result["family_name"])
print("  Format       :", pick_result["format_label"], f"({pick_result['format_id']})")
print("  Persona      :", persona)
print("  Tone         :", tone)
print("  Season label :", pick_result["season_label"])
print("  Topic origin :", pick_result["topic_origin"])
print("  Explanation  :", pick_result["topic_selection_explanation"])
print("  Target length:", f"~{SHORTS_TARGET_SECONDS}s (try stay under ~{SHORTS_SOFT_MAX_SECONDS}s)")
print("  Title limit  :", f"aim <= {SUNO_TITLE_SOFT_MAX_CHARS}, must be <= {SUNO_TITLE_MAX_CHARS}")
print("  Trend pool   :", f"{len(pick_result.get('trending_pool', []))} (preview stored in run_meta)")
print("  Trend prompt :", "USED" if pick_result["trending_prompt_sample"] else "NOT used")
print(f"  Metadata saved to: {meta_path}")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 4. GPT: teaching bullets → dense, rhyming explainer lyrics
#     (pace-aware word budgets to *guide* Shorts staying < ~3 minutes)
#     UPDATED for: general-audience topics explained deeply (intuition-first)
# ─────────────────────────────────────────────────────────────

from typing import List
import json
import re
import math

# Model for lyrics generation
GPT_LYRICS_MODEL = "gpt-4.1"       # stronger for creative writing
LYRICS_LANGUAGE = "Portuguese (Brazil)"        # e.g. "English" or "Danish"

# ─────────────────────────────────────────────────────────────
# Load run metadata from Block 3 (run_meta.json)
# ─────────────────────────────────────────────────────────────

run_meta_path = RUN_DIR / "run_meta.json"
if not run_meta_path.exists():
    raise FileNotFoundError(f"run_meta.json not found at {run_meta_path}. Run Block 3 first.")

run_meta = json.loads(run_meta_path.read_text(encoding="utf-8"))

SHORTS_TARGET_SECONDS = int(run_meta.get("shorts_target_seconds", 120))
SHORTS_SOFT_MAX_SECONDS = int(run_meta.get("shorts_soft_max_seconds", 175))

# Defaults (overridden by pace profiles below; guidance only)
DEFAULT_TARGET_WORDS_MIN = int(run_meta.get("lyrics_target_words_min", 200))
DEFAULT_TARGET_WORDS_MAX = int(run_meta.get("lyrics_target_words_max", 320))
DEFAULT_TARGET_LYRIC_LINES_MIN = int(run_meta.get("lyrics_target_lines_min", 28))
DEFAULT_TARGET_LYRIC_LINES_MAX = int(run_meta.get("lyrics_target_lines_max", 44))

FORMAT_ID = str(run_meta.get("format_id", "") or "").strip()
FORMAT_LABEL = str(run_meta.get("format_label", "") or "").strip()

# Prefer Block 3's explicit pace profile mapping if available
PACE_PROFILE_ID = str(run_meta.get("pace_profile_id", "") or "").strip()
PACE_PROFILE_ID = PACE_PROFILE_ID or str(run_meta.get("lyrics_pace_profile", {}).get("format_id", "") or "").strip()

# ─────────────────────────────────────────────────────────────
# Pace-aware lyric budgets (GUIDANCE, not hard caps)
# ─────────────────────────────────────────────────────────────

PACE_PROFILES = {
    "rap_summary": {
        "wpm_range": (165, 210),
        "instrumental_padding_s": 10,
        "label": "fast / rap-like delivery",
    },
    "science_pop_song": {
        "wpm_range": (135, 175),
        "instrumental_padding_s": 15,
        "label": "medium-fast / upbeat",
    },
    "mindset_diary": {
        "wpm_range": (115, 155),
        "instrumental_padding_s": 20,
        "label": "medium / measured explainer",
    },
    "epic_breakdown": {
        "wpm_range": (95, 135),
        "instrumental_padding_s": 25,
        "label": "slow-medium / cinematic",
    },
    "techwave": {
        "wpm_range": (95, 140),
        "instrumental_padding_s": 25,
        "label": "slow-medium / atmospheric",
    },
    "explain_like_im_5": {
        "wpm_range": (110, 150),
        "instrumental_padding_s": 20,
        "label": "medium / plain-language",
    },
    "__default__": {
        "wpm_range": (120, 165),
        "instrumental_padding_s": 18,
        "label": "default / balanced",
    },
}

def compute_pace_targets(
    pace_key: str,
    shorts_target_s: int,
    shorts_soft_max_s: int,
):
    prof = PACE_PROFILES.get(pace_key) or PACE_PROFILES["__default__"]
    wpm_min, wpm_max = prof["wpm_range"]
    pad = int(prof["instrumental_padding_s"])

    target_total_s = int(shorts_target_s)
    soft_max_total_s = int(shorts_soft_max_s)

    safety_buffer_s = 8
    target_vocals_s = max(60, target_total_s - pad - safety_buffer_s)
    soft_max_vocals_s = max(75, soft_max_total_s - pad - safety_buffer_s)

    words_min = int(math.floor((target_vocals_s / 60.0) * wpm_min))
    words_max = int(math.ceil((soft_max_vocals_s / 60.0) * wpm_max))

    words_min = max(140, min(words_min, 380))
    words_max = max(words_min + 40, min(words_max, 520))

    lines_min = max(20, int(words_min / 10))
    lines_max = max(lines_min + 8, int(words_max / 7))

    return {
        "pace_key": pace_key,
        "profile_label": prof["label"],
        "wpm_range": (wpm_min, wpm_max),
        "instrumental_padding_s": pad,
        "target_vocals_s": target_vocals_s,
        "soft_max_vocals_s": soft_max_vocals_s,
        "target_words_min": words_min,
        "target_words_max": words_max,
        "target_lines_min": lines_min,
        "target_lines_max": lines_max,
    }

PACE_KEY = PACE_PROFILE_ID or FORMAT_ID or "__default__"
pace_targets = compute_pace_targets(PACE_KEY, SHORTS_TARGET_SECONDS, SHORTS_SOFT_MAX_SECONDS)

TARGET_WORDS_MIN = pace_targets["target_words_min"]
TARGET_WORDS_MAX = pace_targets["target_words_max"]
TARGET_LYRIC_LINES_MIN = pace_targets["target_lines_min"]
TARGET_LYRIC_LINES_MAX = pace_targets["target_lines_max"]

print("⏱️ Pace-aware lyric guidance loaded:")
print(f"   - Format: {FORMAT_LABEL or FORMAT_ID or 'unknown'}")
print(f"   - Pace key: {PACE_KEY}")
print(f"   - Pace profile: {pace_targets['profile_label']}")
print(f"   - WPM range: {pace_targets['wpm_range'][0]}–{pace_targets['wpm_range'][1]}")
print(f"   - Reserved instrumental padding: ~{pace_targets['instrumental_padding_s']}s")
print(f"   - Target vocal time: ~{pace_targets['target_vocals_s']}s "
      f"(soft max vocals ~{pace_targets['soft_max_vocals_s']}s)")
print(f"   - Lyric targets: {TARGET_WORDS_MIN}–{TARGET_WORDS_MAX} words, "
      f"{TARGET_LYRIC_LINES_MIN}–{TARGET_LYRIC_LINES_MAX} lyric lines (excluding tags)")

# Persist computed targets into run_meta for debugging / tuning
run_meta["lyrics_target_words_min"] = TARGET_WORDS_MIN
run_meta["lyrics_target_words_max"] = TARGET_WORDS_MAX
run_meta["lyrics_target_lines_min"] = TARGET_LYRIC_LINES_MIN
run_meta["lyrics_target_lines_max"] = TARGET_LYRIC_LINES_MAX
run_meta["lyrics_pace_profile"] = {
    "pace_key": PACE_KEY,
    "format_id": FORMAT_ID,
    "format_label": FORMAT_LABEL,
    "pace_profile_id": PACE_PROFILE_ID,
    "profile_label": pace_targets["profile_label"],
    "wpm_range": list(pace_targets["wpm_range"]),
    "instrumental_padding_s": pace_targets["instrumental_padding_s"],
    "target_vocals_s": pace_targets["target_vocals_s"],
    "soft_max_vocals_s": pace_targets["soft_max_vocals_s"],
}
run_meta_path.write_text(json.dumps(run_meta, indent=2, ensure_ascii=False), encoding="utf-8")

# ─────────────────────────────────────────────────────────────
# Helpers
# ─────────────────────────────────────────────────────────────

TAG_RE = re.compile(r"^\s*\[[^\]]+\]\s*$")

def count_words(text: str) -> int:
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    content_lines = [ln for ln in lines if not TAG_RE.match(ln)]
    joined = " ".join(content_lines)
    tokens = re.findall(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9']+", joined)
    return len(tokens)

def count_lyric_lines(text: str) -> int:
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    return sum(1 for ln in lines if not TAG_RE.match(ln))

def clamp_bullets(bullets: List[str]) -> List[str]:
    bullets = [b.strip("- ").strip() for b in bullets if b.strip()]
    if len(bullets) < 6:
        return bullets
    if len(bullets) > 10:
        return bullets[:10]
    return bullets

def normalize_bullets(bullets: List[str]) -> List[str]:
    out = []
    seen = set()
    for b in bullets:
        b = re.sub(r"\s+", " ", b).strip()
        b = b.rstrip(";")
        bl = b.lower()
        if bl and bl not in seen:
            seen.add(bl)
            out.append(b)
    return out

# ─────────────────────────────────────────────────────────────
# Step 1: teaching bullets (general-audience topic, deep explanation)
# ─────────────────────────────────────────────────────────────

def generate_teaching_bullets(subject: str, language: str = LYRICS_LANGUAGE) -> List[str]:
    """
    Generate 6–10 accurate teaching bullets for a broad audience topic,
    but structured so the song can go deep (intuition → mechanism → example → misconception).
    """
    prompt = (
        f"You are an expert teacher explaining '{subject}' to a general audience in {language}.\n"
        "Assume curiosity, not prior knowledge.\n\n"
        "Rules:\n"
        "- Output ONLY 6 to 10 bullet points.\n"
        "- Each bullet must be ONE sentence, accurate and concrete.\n"
        "- Cover, in order where possible:\n"
        "  (1) what it is (plain words),\n"
        "  (2) the core mechanism (how/why),\n"
        "  (3) a simple everyday example/intuition anchor,\n"
        "  (4) a trade-off/limit/edge case,\n"
        "  (5) a common misconception and the correction.\n"
        "- Define any key term briefly inside the bullet the first time it appears.\n"
        "- Avoid equations unless they are truly essential; prefer concepts.\n"
        "- Do NOT number them, just use '-' at the start of each line.\n"
        "- Avoid claims that require citations, statistics, or named studies.\n"
    )

    resp = openai_client.chat.completions.create(
        model=GPT_LYRICS_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.25,
        max_tokens=700,
    )

    text = resp.choices[0].message.content.strip()
    lines = [line.strip() for line in text.splitlines() if line.strip()]

    bullets = []
    for line in lines:
        if line.startswith("-"):
            line = line[1:].strip()
        bullets.append(line)

    bullets = normalize_bullets(clamp_bullets(bullets))
    if len(bullets) < 6:
        raise ValueError(f"Got too few bullets from GPT: {bullets}")

    return bullets

# ─────────────────────────────────────────────────────────────
# Step 2: bullets → dense, rhyming explainer lyrics (pace-aware)
# ─────────────────────────────────────────────────────────────

def bullets_to_lyrics(
    bullets: List[str],
    subject: str,
    persona: str,
    tone: str,
    language: str = LYRICS_LANGUAGE,
) -> str:
    """
    General-topic, deep explainer lyrics:
    - intuition-first but still dense and accurate
    - defines terms briefly on first use
    - minimal repetition
    - guided by pace-aware word budgets
    """
    bullets_text = "\n".join(f"- {b}" for b in bullets)

    prompt = (
        f"You are writing educational lyrics in {language} for YouTube Shorts.\n"
        f"Audience: general audience (teens+), curious, not experts.\n"
        f"Subject: '{subject}'.\n\n"
        "You MUST cover these teaching points (do not add new claims beyond them):\n"
        f"{bullets_text}\n\n"
        "Lyric style requirements:\n"
        "- Make it Suno-friendly: clean, singable, rhythmic.\n"
        "- Start with intuition (what it is + why it matters), then go mechanism.\n"
        "- When a technical term appears first time, define it in a short, plain line.\n"
        "- Use INTERNAL RHYMES and END RHYMES when natural, but never sacrifice clarity or accuracy.\n"
        "- Keep explanations crisp: prefer short lines; no long multi-clause lines.\n"
        "- MINIMIZE repetition: if you include a hook, repeat it at most once.\n"
        "- Avoid long intros/outros; begin explaining quickly.\n"
        "- Use section tags on their own lines, but keep it simple:\n"
        "  [Intro]\n"
        "  [Explain]\n"
        "  [Explain 2]\n"
        "  [Hook] (optional, short)\n"
        "  [Final]\n\n"
        "Pace + length guidance (do NOT mention numbers in the lyrics):\n"
        f"- Target total lyric words: {TARGET_WORDS_MIN}–{TARGET_WORDS_MAX} (excluding tag lines).\n"
        f"- Target lyric lines: {TARGET_LYRIC_LINES_MIN}–{TARGET_LYRIC_LINES_MAX} (excluding tag lines).\n\n"
        "Musical context:\n"
        f"- Style / vibe: {persona}.\n"
        f"- Tone: {tone}.\n\n"
        "Output rules:\n"
        "- Return ONLY the lyrics with section tags.\n"
        "- No commentary, no bullet points, no emojis.\n"
    )

    resp = openai_client.chat.completions.create(
        model=GPT_LYRICS_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.85,
        max_tokens=1700,
    )

    return resp.choices[0].message.content.strip()

# ─────────────────────────────────────────────────────────────
# Step 3: gentle length correction pass (guidance, not brute force)
# ─────────────────────────────────────────────────────────────

def maybe_adjust_lyrics_to_targets(lyrics: str, bullets: List[str], language: str = LYRICS_LANGUAGE) -> str:
    w = count_words(lyrics)
    n_lines = count_lyric_lines(lyrics)

    too_long = (w > TARGET_WORDS_MAX) or (n_lines > TARGET_LYRIC_LINES_MAX)
    too_short = (w < TARGET_WORDS_MIN) or (n_lines < TARGET_LYRIC_LINES_MIN)

    if not too_long and not too_short:
        return lyrics

    direction = "compress" if too_long else "expand slightly"
    bullets_text = "\n".join(f"- {b}" for b in bullets)

    prompt = (
        f"You are editing educational lyrics in {language}.\n"
        f"Goal: {direction} while keeping clarity for a general audience and keeping musical flow.\n\n"
        "Constraints:\n"
        "- Keep section tags on their own lines.\n"
        "- Keep it Suno-friendly and rhythmic.\n"
        "- Keep light rhyme where natural.\n"
        "- Avoid repetition (no chorus loops).\n"
        "- Do NOT introduce new factual claims.\n"
        "- You MUST still cover these teaching points:\n"
        f"{bullets_text}\n\n"
        "Length guidance (do not mention numbers in the lyrics):\n"
        f"- Target words: {TARGET_WORDS_MIN}–{TARGET_WORDS_MAX} (excluding tag lines)\n"
        f"- Target lines: {TARGET_LYRIC_LINES_MIN}–{TARGET_LYRIC_LINES_MAX} (excluding tag lines)\n\n"
        "Here are the lyrics to revise:\n"
        f"{lyrics}\n\n"
        "Return ONLY the revised lyrics."
    )

    resp = openai_client.chat.completions.create(
        model=GPT_LYRICS_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6,
        max_tokens=1700,
    )

    return resp.choices[0].message.content.strip()

# ─────────────────────────────────────────────────────────────
# Run the lyrics pipeline for the current run
# ─────────────────────────────────────────────────────────────

print("🧠 Generating teaching bullets (intuition-first, deep)...")
bullets = generate_teaching_bullets(subject, language=LYRICS_LANGUAGE)

bullets_path = RUN_DIR / "logs" / "teaching_bullets.txt"
bullets_path.write_text("\n".join(f"- {b}" for b in bullets), encoding="utf-8")

print("✅ Bullets generated:")
for b in bullets:
    print("  -", b)
print(f"   Saved to: {bullets_path}")

print("\n🎼 Generating pace-aware explainer lyrics (general topic, deep explanation)...")
lyrics_raw = bullets_to_lyrics(bullets, subject, persona, tone, language=LYRICS_LANGUAGE)
lyrics = maybe_adjust_lyrics_to_targets(lyrics_raw, bullets=bullets, language=LYRICS_LANGUAGE)

final_words = count_words(lyrics)
final_lines = count_lyric_lines(lyrics)
print(f"\n📏 Lyric stats (excluding tags): {final_words} words, {final_lines} lyric lines")
print(f"🎚️ Pace profile used: {pace_targets['profile_label']} (pace_key={PACE_KEY})")

lyrics_path = RUN_DIR / "lyrics.txt"
lyrics_path.write_text(lyrics, encoding="utf-8")

print("✅ Lyrics generated and saved:")
print(f"   {lyrics_path}")
print("\n──── Sample of lyrics (first ~800 characters) ────")
print(lyrics[:800])
print("\n(Full lyrics are saved to lyrics.txt in the run folder.)")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 5. Suno Custom Mode: generate song from lyrics and download
# ─────────────────────────────────────────────────────────────

import requests
import time

# Use your known-working base + model
SUNO_BASE = config.suno_base_url.rstrip("/")  # from Block 1 config
SUNO_MODEL = "V5"                             # adjust if you want another Suno model

def suno_headers():
    return {
        "Authorization": f"Bearer {config.suno_api_key}",
        "Content-Type": "application/json"
    }

def suno_generate_custom(lyrics: str, subject: str, persona: str, model: str = SUNO_MODEL) -> str:
    """
    Custom mode: we provide full lyrics, and Suno sings them in the given style.
    Returns a taskId we can poll.
    """
    # Keep title short & YouTube-friendly – can be improved later
    title = f"{subject.title()} explained in a song"
    # Use persona as style hint
    style = persona

    payload = {
        "customMode": True,
        "instrumental": False,   # we want vocals singing our lyrics
        "model": model,
        "title": title[:100],    # avoid overly long titles
        "style": style[:200],
        "prompt": lyrics,        # our generated lyrics become the sung text

        # Your Suno provider requires a callBackUrl field in the payload,
        # even if we don't actually use webhooks and instead poll manually.
        "callBackUrl": "https://example.com/callback"
    }

    print("🎵 Sending Custom Mode generate request to Suno...")
    r = requests.post(f"{SUNO_BASE}/generate", headers=suno_headers(), json=payload, timeout=60)
    r.raise_for_status()
    j = r.json()

    # Many Suno-compatible APIs return { code: 200, data: { taskId: ... } }
    if j.get("code") != 200:
        raise RuntimeError(f"Suno generate error: {j}")

    task_id = j["data"]["taskId"]
    print("✅ Suno task created:", task_id)
    return task_id

def suno_record_info(task_id: str):
    """
    Query Suno for the status and metadata of a generation task.
    """
    r = requests.get(
        f"{SUNO_BASE}/generate/record-info",
        headers=suno_headers(),
        params={"taskId": task_id},
        timeout=60,
    )
    r.raise_for_status()
    return r.json()

def wait_for_suno_song(task_id: str, timeout_s: int = 360, poll_every: int = 5):
    """
    Poll the Suno task until it finishes or times out.
    Returns the final record-info JSON.
    """
    print("⏳ Waiting for Suno to finish generation...")
    t0 = time.time()
    status = "PENDING"
    last = None

    while time.time() - t0 < timeout_s:
        last = suno_record_info(task_id)
        data = last.get("data", {})
        status = data.get("status")
        print(f"  - Status: {status}")

        if status in ("SUCCESS", "FIRST_SUCCESS"):
            print("✅ Suno generation completed.")
            return last

        time.sleep(poll_every)

    raise TimeoutError(f"Suno task timeout. Last status={status}, payload={last}")


def download_file(url: str, out_path: pathlib.Path):
    """
    Download a file from URL to out_path using streaming.
    """
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with requests.get(url, stream=True, timeout=120) as r:
        r.raise_for_status()
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)


def get_track_with_audio(task_id: str, initial_rec: dict, max_retries: int = 5, retry_delay: int = 5):
    """
    Some Suno tenants mark status SUCCESS before audioUrl is actually populated.
    This helper:
      - Looks for the first track with a non-empty audioUrl
      - If not found, re-polls record-info a few times with a delay
    """
    rec = initial_rec
    for attempt in range(max_retries + 1):
        data = rec.get("data", {})
        response = data.get("response", {})
        suno_data = response.get("sunoData", []) or []

        tracks_with_url = [t for t in suno_data if t.get("audioUrl")]
        if tracks_with_url:
            if attempt > 0:
                print(f"✅ Audio URL available after {attempt} extra check(s).")
            return tracks_with_url[0]

        if attempt < max_retries:
            print(f"⚠️ audioUrl not ready yet (attempt {attempt+1}/{max_retries}), retrying...")
            time.sleep(retry_delay)
            rec = suno_record_info(task_id)
        else:
            break

    raise RuntimeError(f"No track with a valid audioUrl found after retries. Last payload: {rec}")


# ── Run Suno generation for this run ────────────────────────

# Load lyrics from the file we saved in Block 4
lyrics_path = RUN_DIR / "lyrics.txt"
if not lyrics_path.exists():
    raise FileNotFoundError(f"lyrics.txt not found at {lyrics_path}. Run Block 4 first.")

lyrics_text = lyrics_path.read_text(encoding="utf-8")

# 1) Create the generation task
task_id = suno_generate_custom(lyrics_text, subject=subject, persona=persona, model=SUNO_MODEL)

# 2) Poll until the song is ready
rec = wait_for_suno_song(task_id)

# 3) Extract track info with robust audioUrl handling
track = get_track_with_audio(task_id, rec)
AUDIO_URL = track["audioUrl"]
AUDIO_ID  = track["id"]
TITLE     = track.get("title", "Untitled")
DURATION  = track.get("duration", None)

# 4) Download audio into this run's audio folder
audio_path = RUN_DIR / "audio" / "song.mp3"
download_file(AUDIO_URL, audio_path)

# 5) Save metadata for later steps (subtitles, video, upload)
song_meta = {
    "task_id": task_id,
    "audio_id": AUDIO_ID,
    "title": TITLE,
    "duration": DURATION,
    "audio_url": AUDIO_URL,
    "subject": subject,
    "persona": persona,
    "tone": tone,
}

song_meta_path = RUN_DIR / "logs" / "suno_song_meta.json"
song_meta_path.write_text(json.dumps(song_meta, indent=2, ensure_ascii=False), encoding="utf-8")

print("🎧 Suno song ready!")
print("  Title   :", TITLE)
print("  AudioID :", AUDIO_ID)
print("  Duration:", DURATION)
print("  Audio   :", audio_path)
print(f"  Meta    : {song_meta_path}")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 6. Timestamped lyrics (Suno) → word groups for subtitles
#    (Retry + validation; max wait ~5 minutes)
# ─────────────────────────────────────────────────────────────

import json
import random
import time
import re

# We reuse:
# - SUNO_BASE
# - suno_headers()
# - RUN_DIR
# - requests

# Subtitle grouping parameters:
# - More words per caption for readability
# - Also use a rough max duration per caption
CAP_WORDS_MIN = 5
CAP_WORDS_MAX = 10
CAP_MAX_GAP   = 0.25   # max silence gap (seconds) allowed inside one caption
CAP_TARGET_MAX_DUR = 3.0  # seconds; aim to keep each caption under this

# ─────────────────────────────────────────────────────────────
# Retry settings
# ─────────────────────────────────────────────────────────────
LYRICS_MAX_WAIT_S = 300          # total cap: 5 minutes
LYRICS_RETRY_DELAY_S = 8         # base delay between tries
LYRICS_RETRY_JITTER_S = 4        # add 0–4s randomness
LYRICS_REQUEST_TIMEOUT_S = 60    # timeout per request
LYRICS_MIN_WORDS_OK = 40         # sanity check: must look like real alignment


def suno_aligned_words_once(task_id: str, audio_id: str):
    """
    Single attempt: call Suno timestamped lyrics endpoint.
    Returns (alignedWords_or_None, raw_json).
    """
    url = f"{SUNO_BASE}/generate/get-timestamped-lyrics"
    payload = {"taskId": task_id, "audioId": audio_id}

    r = requests.post(url, headers=suno_headers(), json=payload, timeout=LYRICS_REQUEST_TIMEOUT_S)
    r.raise_for_status()
    j = r.json()

    aligned = j.get("data", {}).get("alignedWords")
    if j.get("code") == 200 and aligned:
        return aligned, j

    return None, j


def aligned_words_to_WORDS(aligned):
    """
    Convert Suno alignedWords to our WORDS format.
    Filters out empty words and coerces types safely.
    """
    words = []
    for a in aligned or []:
        w = str(a.get("word", "")).strip()
        if not w:
            continue
        if not a.get("success", True):
            continue
        try:
            start = float(a.get("startS"))
            end   = float(a.get("endS"))
        except Exception:
            continue
        if end <= start:
            continue
        words.append({"w": w, "start": start, "end": end})
    return words


def words_look_valid(words) -> bool:
    """
    Heuristics to confirm we actually got real aligned lyrics.
    Prevents 'empty' or partial junk from passing.
    """
    if not words or len(words) < LYRICS_MIN_WORDS_OK:
        return False

    # Basic monotonic time sanity check
    last_end = -1.0
    bad = 0
    for x in words[:200]:  # sample first 200 for speed
        if x["start"] < 0 or x["end"] < 0:
            bad += 1
        if x["start"] < last_end - 0.05:
            bad += 1
        last_end = max(last_end, x["end"])

    if bad > 5:
        return False

    # Must contain some alphabetic content overall (avoid weird non-lyrics payload)
    sample_text = " ".join(w["w"] for w in words[:60])
    if not any(ch.isalpha() for ch in sample_text):
        return False

    return True


def fetch_suno_words_with_retry(task_id: str, audio_id: str, max_wait_s: int = LYRICS_MAX_WAIT_S):
    """
    Keep trying until we get valid WORDS or we hit max_wait_s.
    Returns (WORDS_or_None, debug_info_dict).
    """
    t0 = time.time()
    attempt = 0
    last_response_preview = None

    print("📝 Fetching Suno timestamped lyrics (retry up to ~5 minutes)...")

    while True:
        attempt += 1
        elapsed = int(time.time() - t0)

        if elapsed >= max_wait_s:
            print(f"⏭️ Gave up after {elapsed}s (max {max_wait_s}s). Proceeding WITHOUT subtitles.")
            return None, {
                "attempts": attempt - 1,
                "elapsed_s": elapsed,
                "status": "timeout_no_valid_alignment",
                "last_response_preview": last_response_preview,
            }

        try:
            aligned, raw = suno_aligned_words_once(task_id, audio_id)

            last_response_preview = {
                "code": raw.get("code"),
                "msg": raw.get("msg") or raw.get("message"),
                "has_alignedWords": bool(raw.get("data", {}).get("alignedWords")),
            }

            if aligned:
                words = aligned_words_to_WORDS(aligned)
                if words_look_valid(words):
                    print(f"✅ Valid aligned lyrics received on attempt {attempt} (elapsed ~{elapsed}s).")
                    return words, {
                        "attempts": attempt,
                        "elapsed_s": elapsed,
                        "status": "success",
                        "last_response_preview": last_response_preview,
                    }
                else:
                    print(f"⚠️ Attempt {attempt}: alignedWords returned but failed validation (len={len(words)}).")
            else:
                print(f"⚠️ Attempt {attempt}: no alignedWords yet (elapsed ~{elapsed}s).")

        except Exception as e:
            print(f"⚠️ Attempt {attempt}: request failed: {e}")

        # Wait before retry (jittered), without oversleeping past max_wait_s
        sleep_s = LYRICS_RETRY_DELAY_S + random.randint(0, LYRICS_RETRY_JITTER_S)
        remaining = max_wait_s - (time.time() - t0)
        sleep_s = min(sleep_s, max(0, remaining))
        if sleep_s > 0:
            time.sleep(sleep_s)


def group_words(words, min_k=5, max_k=10, max_gap=0.25, target_max_dur=3.0):
    """
    Group word-level timestamps into longer subtitle chunks.

    Goals:
    - 5–10 words per caption
    - Don't cross large gaps of silence (> max_gap)
    - Try to keep each caption under target_max_dur seconds
    """
    groups = []
    i = 0
    n = len(words)

    while i < n:
        start_time = words[i]["start"]
        j = i + 1

        while j < n:
            num = j - i
            current_end = words[j - 1]["end"]
            dur = current_end - start_time
            gap_to_next = words[j]["start"] - words[j - 1]["end"] if j < n else 0.0

            if num >= max_k:
                break
            if dur >= target_max_dur:
                break
            if gap_to_next > max_gap:
                break

            j += 1

        while (j < n) and ((j - i) < min_k):
            gap_to_next = words[j]["start"] - words[j - 1]["end"]
            if gap_to_next > max_gap:
                break
            j += 1

        chunk = words[i:j]
        txt = " ".join(w["w"] for w in chunk)
        start = chunk[0]["start"]
        end = chunk[-1]["end"]
        groups.append({"text": txt, "start": start, "end": end})

        i = j

    return groups


# ── Load Suno song metadata (from Block 5) ───────────────────

song_meta_path = RUN_DIR / "logs" / "suno_song_meta.json"
if not song_meta_path.exists():
    raise FileNotFoundError(f"suno_song_meta.json not found at {song_meta_path}. Run Block 5 first.")

song_meta = json.loads(song_meta_path.read_text(encoding="utf-8"))
task_id = song_meta["task_id"]
audio_id = song_meta["audio_id"]

# ── Fetch aligned words from Suno (with retry) ────────────────

WORDS, lyrics_debug = fetch_suno_words_with_retry(task_id, audio_id, max_wait_s=LYRICS_MAX_WAIT_S)

# Save debug info always
debug_path = RUN_DIR / "logs" / "aligned_words_fetch_debug.json"
debug_path.write_text(json.dumps(lyrics_debug, indent=2, ensure_ascii=False), encoding="utf-8")
print(f"🧾 Saved fetch debug info to: {debug_path}")

# Save raw aligned words (if any) for debugging
aligned_words_path = RUN_DIR / "logs" / "aligned_words.json"
aligned_words_path.write_text(json.dumps(WORDS or [], indent=2, ensure_ascii=False), encoding="utf-8")
print(f"💾 Saved raw aligned words to: {aligned_words_path}")

# ── Group into subtitle bursts ───────────────────────────────

GROUPS = []
if WORDS:
    GROUPS = group_words(
        WORDS,
        min_k=CAP_WORDS_MIN,
        max_k=CAP_WORDS_MAX,
        max_gap=CAP_MAX_GAP,
        target_max_dur=CAP_TARGET_MAX_DUR,
    )
    print(f"🟨 Groups created: {len(GROUPS)} (first few):")
    for g in GROUPS[:3]:
        print(f"  - [{g['start']:.2f}–{g['end']:.2f}] {g['text']}")
else:
    print("⚠️ No valid WORDS available, GROUPS will be empty and subtitles will be skipped.")

groups_path = RUN_DIR / "logs" / "groups.json"
groups_path.write_text(json.dumps(GROUPS, indent=2, ensure_ascii=False), encoding="utf-8")
print(f"💾 Saved grouped subtitles to: {groups_path}")


In [ ]:
# ─────────────────────────────────────────────────────────────
# 7. Write ASS subtitles with glow + shadow (no dark bar)
# ─────────────────────────────────────────────────────────────

from pathlib import Path
import math
import json

ASS_PATH = RUN_DIR / "subs.ass"

# Ensure video constants exist (so Block 7 can run even if earlier cells didn’t define them)
if "VIDEO_W" not in globals():
    VIDEO_W = 1080
if "VIDEO_H" not in globals():
    VIDEO_H = 1920
if "FPS" not in globals():
    FPS = 30

# Ensure GROUPS exists (created in the earlier lyric→timing grouping step)
if "GROUPS" not in globals():
    raise NameError("GROUPS is not defined. Run the block that creates subtitle GROUPS before Block 7.")

def ass_time(t: float) -> str:
    # ASS expects H:MM:SS.CS (centiseconds)
    if t < 0:
        t = 0
    cs = int(round((t - math.floor(t)) * 100))
    sec = int(t) % 60
    m = (int(t) // 60) % 60
    h = int(t) // 3600
    if cs == 100:
        cs = 0
        sec += 1
    return f"{h}:{m:02d}:{sec:02d}.{cs:02d}"

# ─────────────────────────────────────────────────────────────
# Single-layer subtitle style (white text with glow + shadow)
# ─────────────────────────────────────────────────────────────

ASS_HEADER = f"""[Script Info]
ScriptType: v4.00+
PlayResX: {VIDEO_W}
PlayResY: {VIDEO_H}
ScaledBorderAndShadow: yes

[V4+ Styles]
Format: Name,Fontname,Fontsize,PrimaryColour,SecondaryColour,OutlineColour,BackColour,Bold,Italic,Underline,StrikeOut,ScaleX,ScaleY,Spacing,Angle,BorderStyle,Outline,Shadow,Alignment,MarginL,MarginR,MarginV,Encoding
Style: FG,DejaVu Sans,56,&H00FFFFFF,&H00FFFFFF,&H00000000,&H32000000,-1,0,0,0,100,100,0,0,1,4,6,5,5,30,30,20,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""

def write_ass(groups):
    lines = [ASS_HEADER]

    for g in groups:
        text = str(g["text"]).replace("\n", " ").strip()
        start = ass_time(float(g["start"]))
        end = ass_time(float(g["end"]))

        # Foreground glowing text only (no BG bar)
        fg_line = (
            f"Dialogue: 0,{start},{end},FG,,0,0,40,,"
            f"{{\\an5\\bord4\\blur2\\shad2}}{text}"
        )
        lines.append(fg_line)

    ASS_PATH.write_text("\n".join(lines), encoding="utf-8")

# Clamp cue timings and validate
for g in GROUPS:
    g["start"] = max(0.0, float(g["start"]))
    g["end"] = float(g["end"])

GROUPS = [g for g in GROUPS if g["end"] > g["start"] + 0.05]

write_ass(GROUPS)
print("✨ ASS subtitles (glow only, no bar) written to:", ASS_PATH)

# preview first lines
print("\n".join(ASS_PATH.read_text(encoding="utf-8").splitlines()[:20]))


In [ ]:
# ─────────────────────────────────────────────────────────────
# 8. Generate background image (OpenAI) + render dynamic MP4
#    - Gentle zoom in / out loop
#    - ASS subtitles overlaid via `subtitles` filter
# ─────────────────────────────────────────────────────────────

import base64
import subprocess
import json
import math

# We reuse:
# - openai_client
# - subject, persona, tone, LYRICS_LANGUAGE
# - RUN_DIR
# - audio_path (from Block 5)
# - ASS_PATH, VIDEO_W, VIDEO_H, FPS (from Block 7)

BACKGROUND_IMG_PATH = RUN_DIR / "images" / "background.png"
FINAL_MP4 = RUN_DIR / "video" / "shorts_final.mp4"

# Sanity checks
if not audio_path.exists():
    raise FileNotFoundError(f"Audio file not found at {audio_path}. Run Block 5 first.")
if not ASS_PATH.exists():
    raise FileNotFoundError(f"ASS subtitles not found at {ASS_PATH}. Run Block 7 first.")

# Try to get audio duration from Suno meta (not strictly needed for zoom, but useful to log)
song_meta_path = RUN_DIR / "logs" / "suno_song_meta.json"
if song_meta_path.exists():
    song_meta = json.loads(song_meta_path.read_text(encoding="utf-8"))
    audio_duration = song_meta.get("duration", None)
else:
    audio_duration = None

if not audio_duration or audio_duration <= 0:
    audio_duration = 90.0  # soft default for logging / future use

def build_background_prompt(subject: str, persona: str, tone: str, language: str = LYRICS_LANGUAGE) -> str:
    """
    Build a descriptive prompt for a portrait background image.
    """
    return (
        f"Vertical illustration about {subject} in the style of {persona}, "
        f"with a {tone} mood. High quality, cinematic, soft lighting, "
        "no text, no logos, no UI, no watermarks. Composition leaves free space in the center "
        "so subtitles are readable."
    )

def generate_background_image(prompt: str, out_path: pathlib.Path):
    """
    Use OpenAI image model to generate a portrait background image.
    Valid sizes: 1024x1024, 1024x1536, 1536x1024, auto.
    """
    print("🖼 Generating background image with OpenAI...")

    response = openai_client.images.generate(
        model="gpt-image-1",
        prompt=prompt,
        size="1024x1536",   # portrait ratio
        n=1
    )

    b64 = response.data[0].b64_json
    img_bytes = base64.b64decode(b64)

    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_path.write_bytes(img_bytes)
    print(f"✅ Background image saved to: {out_path}")

# ── 1) Generate background image ─────────────────────────────

bg_prompt = build_background_prompt(subject, persona, tone)
print("🧾 Background prompt:")
print(" ", bg_prompt)

generate_background_image(bg_prompt, BACKGROUND_IMG_PATH)

# ── 2) Render final vertical MP4 with gentle zoom in/out + subtitles ─────

# We:
# - scale the source a bit larger than the final frame
# - apply zoompan with an oscillating zoom:
#     zoom(t) ≈ 1.0 → 1.07 → 1.0 → ...
#   using frame number `on` (zoompan's output frame index)
# - keep the image centered while zooming
# - overlay ASS subtitles with the `subtitles` filter

# Make the base image a bit larger so we can zoom >1.0 without quality loss
BASE_W = int(VIDEO_W * 1.2)
BASE_H = int(VIDEO_H * 1.2)

# Zoom expression:
# Centered at ~1.035 with amplitude 0.035 → range ~1.0 to ~1.07 (5–7% zoom)
zoom_expr = "1.035+0.035*sin(on/120)"

# Center the zoom around the middle of the frame
x_expr = "iw/2 - iw/zoom/2"
y_expr = "ih/2 - ih/zoom/2"

# Build the filter_complex string
filter_complex = (
    f"[0:v]"
    f"scale={BASE_W}:{BASE_H},"
    f"zoompan="
        f"z='{zoom_expr}':"
        f"x='{x_expr}':"
        f"y='{y_expr}':"
        f"d=1:"
        f"s={VIDEO_W}x{VIDEO_H}:"
        f"fps={FPS},"
    f"subtitles='{ASS_PATH.as_posix()}'"
    f"[v]"
)

ffmpeg_cmd = f"""
ffmpeg -y -loop 1 -framerate {FPS} -i "{BACKGROUND_IMG_PATH}" -i "{audio_path}" \
  -filter_complex "{filter_complex}" \
  -map "[v]" -map 1:a \
  -c:v libx264 -preset veryfast -pix_fmt yuv420p \
  -c:a aac -b:a 192k -shortest "{FINAL_MP4}"
"""

print("\n🎬 Running ffmpeg to create final video with zooming background + subtitles:")
print(ffmpeg_cmd)

process = subprocess.run(
    ["bash", "-lc", ffmpeg_cmd],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

print("── ffmpeg output ──")
print(process.stdout)

if process.returncode != 0:
    raise RuntimeError(f"ffmpeg failed with exit code {process.returncode}")

print("\n✅ Final Shorts video with zooming background + subtitles created!")
print("   Path:", FINAL_MP4)


In [ ]:
# ─────────────────────────────────────────────────────────────
# 9. Generate high-quality metadata
#    - A/B title candidates + heuristic scoring
#    - A/B description candidates + heuristic scoring
#    - Refined tags
#    - Pinned comment
# ─────────────────────────────────────────────────────────────

import json
import re

# Configurable series branding: this will be appended to every chosen title
SERIES_SUFFIX = " | Aprenda com Ritmo"  # tweak this to your channel brand later

METADATA_JSON_PATH = RUN_DIR / "logs" / "metadata.json"

# Load lyrics & song meta
lyrics_path = RUN_DIR / "lyrics.txt"
song_meta_path = RUN_DIR / "logs" / "suno_song_meta.json"

if not lyrics_path.exists():
    raise FileNotFoundError(f"lyrics.txt not found at {lyrics_path}. Run Block 4 first.")
if not song_meta_path.exists():
    raise FileNotFoundError(f"suno_song_meta.json not found at {song_meta_path}. Run Block 5 first.")

lyrics_text = lyrics_path.read_text(encoding="utf-8")
song_meta = json.loads(song_meta_path.read_text(encoding="utf-8"))

duration_sec = song_meta.get("duration", None)

# ─────────────────────────────────────────────────────────────
# Helpers: subject keywords
# ─────────────────────────────────────────────────────────────

def extract_subject_keywords(subject: str) -> list[str]:
    """
    Extract simple keywords from the subject for matching.
    Very lightweight: split on spaces, drop super common words.
    """
    stopwords = {"the", "a", "an", "of", "and", "for", "to", "in", "vs", "very", "high", "level", "high-level"}
    tokens = [w.lower() for w in re.split(r"[^a-zA-Z0-9]+", subject) if w]
    return [t for t in tokens if t not in stopwords]


# ─────────────────────────────────────────────────────────────
# Title scoring heuristics for A/B testing
# ─────────────────────────────────────────────────────────────

def score_title(title: str, subject: str) -> float:
    """
    Simple heuristic score:
    - Reward:
      - Contains subject keywords
      - Reasonable length (<80 chars, sweet spot ~40–70)
      - Contains benefit words like 'explained', 'finally', 'simple'
      - Contains a question mark
    - Penalize:
      - Hashtags
      - Overly long
      - All caps or spammy punctuation
    """
    score = 0.0
    t = title.strip()
    tl = t.lower()

    # Length
    length = len(t)
    if 40 <= length <= 72:
        score += 3.0
    elif length <= 90:
        score += 1.5
    else:
        score -= 2.0

    # Subject keyword coverage
    subject_keywords = extract_subject_keywords(subject)
    kw_matches = sum(1 for kw in subject_keywords if kw in tl)
    score += min(kw_matches * 1.5, 4.0)  # cap keyword bonus

    # Benefit / clarity words
    benefit_words = [
        "explained", "explained simply", "finally makes sense",
        "for beginners", "made simple", "in plain language",
        "in simple terms", "under 60 seconds", "easy breakdown",
        "learn fast", "demystified", "step by step", "the basics"
    ]
    for bw in benefit_words:
        if bw in tl:
            score += 2.0
            break  # don't over-count

    # Question mark can increase curiosity
    if "?" in t:
        score += 1.0

    # Penalize hashtags or emojis
    if "#" in t:
        score -= 3.0
    if any(ch in t for ch in ["🔥", "💰", "🚀", "🤯", "😱"]):
        score -= 2.0

    # Penalize shouty ALL CAPS (ignoring small words)
    letters_only = re.sub(r"[^A-Za-z]+", "", t)
    if letters_only and letters_only.isupper():
        score -= 3.0

    # Penalize too many ! or ? overall
    if t.count("!") + t.count("?") > 3:
        score -= 1.5

    return score


def pick_best_title(candidates: list[str], subject: str) -> str:
    """
    Score all candidates and pick the best one.
    """
    if not candidates:
        raise ValueError("No title candidates provided.")

    scored = []
    for c in candidates:
        c_clean = c.strip()
        if not c_clean:
            continue
        s = score_title(c_clean, subject)
        scored.append((s, c_clean))

    if not scored:
        raise ValueError("All title candidates were empty or invalid.")

    scored.sort(key=lambda x: x[0], reverse=True)
    best_score, best_title = scored[0]

    print("🏆 Title A/B ranking:")
    for s, t in scored:
        mark = " <– chosen" if t == best_title else ""
        print(f"  [{s:+.2f}] {t}{mark}")

    return best_title


# ─────────────────────────────────────────────────────────────
# Description scoring heuristics (for A/B testing)
# ─────────────────────────────────────────────────────────────

def score_description(desc: str, subject: str) -> float:
    """
    Heuristic description score:
    - Hook in the first 1–2 sentences
    - Contains subject keywords
    - Contains CTA words (like, comment, subscribe)
    - Ends with a question
    - Hashtags in a block near the end
    - Reasonable total length (not too short, not a wall of text)
    """
    score = 0.0
    d = desc.strip()
    dl = d.lower()

    # Rough sentence split
    sentences = re.split(r"(?<=[.!?])\s+", d)
    first_sentence = sentences[0] if sentences else d
    first_len = len(first_sentence)

    # Hook length
    if 40 <= first_len <= 200:
        score += 3.0
    elif first_len <= 260:
        score += 1.0
    else:
        score -= 1.0

    # Subject keywords
    subject_keywords = extract_subject_keywords(subject)
    kw_matches = sum(1 for kw in subject_keywords if kw in dl)
    score += min(kw_matches * 1.0, 4.0)

    # CTA presence
    for word in ["like", "comment", "subscribe", "share"]:
        if word in dl:
            score += 1.5
            break

    # Question at the end (for engagement)
    tail = d[-200:]
    if "?" in tail:
        score += 2.0

    # Hashtags near the end, not spammed at the start
    hash_positions = [m.start() for m in re.finditer(r"#", d)]
    if hash_positions:
        # Check if majority of hashtags are in last 1/3 of text
        last_third = len(d) * (2.0 / 3.0)
        tail_hashes = sum(1 for pos in hash_positions if pos >= last_third)
        if tail_hashes >= max(1, len(hash_positions) // 2):
            score += 2.0
        else:
            score -= 1.0

    # Overall length
    total_len = len(d)
    if 200 <= total_len <= 1500:
        score += 2.0
    elif total_len > 2500:
        score -= 1.5

    return score


def pick_best_description(candidates: list[str], subject: str) -> str:
    """
    Score all description candidates and pick the best one.
    """
    if not candidates:
        raise ValueError("No description candidates provided.")

    scored = []
    for c in candidates:
        c_clean = c.strip()
        if not c_clean:
            continue
        s = score_description(c_clean, subject)
        scored.append((s, c_clean))

    if not scored:
        raise ValueError("All description candidates were empty or invalid.")

    scored.sort(key=lambda x: x[0], reverse=True)
    best_score, best_desc = scored[0]

    print("🏆 Description A/B ranking (scores only):")
    for s, _ in scored:
        mark = " <– chosen" if _ == best_desc else ""
        print(f"  [{s:+.2f}]{mark}")

    return best_desc


# ─────────────────────────────────────────────────────────────
# GPT metadata generator (multi-title + multi-description)
# ─────────────────────────────────────────────────────────────

def generate_video_metadata(
    subject: str,
    persona: str,
    tone: str,
    lyrics: str,
    duration_sec: float | None,
    language: str = LYRICS_LANGUAGE,
) -> dict:
    """
    Use GPT to generate:
    - multiple title candidates (for A/B testing)
    - multiple description candidates (for A/B testing)
    - tags (SEO keywords, no '#')
    - pinned_comment (for manual pinning to boost engagement)
    Returns a dict with:
      - title: chosen best title (with suffix)
      - title_candidates: [... all raw candidates ...]
      - description: chosen best description
      - description_candidates: [...]
      - tags: refined tags
      - pinned_comment: str
    """
    duration_hint = ""
    if duration_sec:
        duration_hint = f"The song is about {int(duration_sec)} seconds long. "

    prompt = (
        f"You are an expert YouTube Shorts growth strategist and copywriter.\n\n"
        f"Video details:\n"
        f"- Language: {language}\n"
        f"- Subject: {subject}\n"
        f"- Musical persona/style: {persona}\n"
        f"- Emotional tone: {tone}\n"
        f"- {duration_hint}\n"
        "The video is an educational song that explains the subject in a fun, catchy way.\n\n"
        "You will receive the full song lyrics. Use them ONLY as inspiration for tone and angle, "
        "not to copy line-by-line.\n\n"
        "Your job:\n"
        "1) Create 5 STRONG, professional-looking TITLE CANDIDATES that:\n"
        "   - Feel like a human made them, not AI.\n"
        "   - Do NOT contain hashtags or emojis.\n"
        "   - Put the main keyword (subject) near the beginning.\n"
        "   - Each uses a slightly different angle (curiosity, benefit, emotional, question, etc.).\n"
        f"   - Each should be written WITHOUT the series suffix. I will append \"{SERIES_SUFFIX}\" myself.\n"
        "   - Maximum ~80 characters each.\n"
        "2) Create 3 DESCRIPTION CANDIDATES that:\n"
        "   - Each starts with a strong 1–2 sentence hook about the subject and what viewers will learn.\n"
        "   - Each includes a short paragraph (3–5 sentences) summarizing what the song explains.\n"
        "   - Each includes a call to action to LIKE, COMMENT and SUBSCRIBE.\n"
        "   - Each ends with a question that invites comments (e.g., what should we cover next?).\n"
        "   - After the explanatory text, add a blank line, then a block of 10–20 relevant HASHTAGS "
        "(with '#', for example #shorts #education #physics). Include #shorts and #youtubeshorts.\n"
        "   - The descriptions can vary the hook angle (curiosity, relatable pain, promise of clarity, etc.).\n"
        "3) Create a TAGS list (YouTube 'tags' / keywords, NOT hashtags):\n"
        "   - 20–30 items.\n"
        "   - No '#' characters.\n"
        "   - Mix of exact subject phrases, search-style queries, and general edutainment keywords.\n"
        "   - All in the same language as the titles/descriptions.\n"
        "4) Create a PINNED COMMENT suggestion:\n"
        "   - 1–3 short sentences.\n"
        "   - Encourages viewers to comment what topic they want songified next.\n"
        "   - Optionally references a key question from the video.\n"
        "   - No hashtags.\n\n"
        "VERY IMPORTANT:\n"
        f"- Title candidates: no hashtags, no emojis, do NOT include the suffix \"{SERIES_SUFFIX}\".\n"
        "- Descriptions: hashtags only at the end in a block, not in the opening hook.\n"
        "- Tags: no '#', just plain keywords.\n\n"
        "Return your answer as a VALID JSON object with this exact structure:\n"
        "{\n"
        "  \"title_candidates\": [\"...\", \"...\", \"...\", \"...\", \"...\"],\n"
        "  \"description_candidates\": [\"...\", \"...\", \"...\"],\n"
        "  \"tags\": [\"tag1\", \"tag2\", \"tag3\", ...],\n"
        "  \"pinned_comment\": \"...\"\n"
        "}\n\n"
        "Here are the lyrics for context:\n"
        "----- LYRICS START -----\n"
        f"{lyrics}\n"
        "----- LYRICS END -----\n"
    )

    resp = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,
        max_tokens=1500,
    )

    raw = resp.choices[0].message.content.strip()

    # Try to parse as JSON
    try:
        meta = json.loads(raw)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", raw, re.DOTALL)
        if not match:
            raise ValueError(f"Metadata response was not valid JSON:\n{raw}")
        meta = json.loads(match.group(0))

    # Basic sanity checks
    if "tags" not in meta:
        raise ValueError(f"Metadata JSON missing required field 'tags': {meta.keys()}")

    # ── Titles: get candidates, clean, pick best ──────────────
    title_candidates = meta.get("title_candidates") or []
    if isinstance(title_candidates, str):
        title_candidates = [title_candidates]
    if not title_candidates and "title" in meta:
        title_candidates = [meta["title"]]

    cleaned_title_candidates = []
    for c in title_candidates:
        c_str = str(c).strip()
        if not c_str:
            continue
        c_str = c_str.replace("{SERIES_SUFFIX}", "").strip()
        c_str = c_str.replace("#", "").strip()
        cleaned_title_candidates.append(c_str)

    if not cleaned_title_candidates:
        raise ValueError("No usable title candidates after cleaning.")

    chosen_title = pick_best_title(cleaned_title_candidates, subject)

    # Append series suffix if not already present
    if SERIES_SUFFIX.strip().lower() not in chosen_title.lower():
        final_title = f"{chosen_title}{SERIES_SUFFIX}"
    else:
        final_title = chosen_title

    # ── Descriptions: get candidates, pick best ───────────────
    desc_candidates = meta.get("description_candidates") or []
    if isinstance(desc_candidates, str):
        desc_candidates = [desc_candidates]
    if not desc_candidates and "description" in meta:
        desc_candidates = [meta["description"]]

    cleaned_desc_candidates = []
    for d in desc_candidates:
        d_str = str(d).strip()
        if not d_str:
            continue
        cleaned_desc_candidates.append(d_str)

    if not cleaned_desc_candidates:
        raise ValueError("No usable description candidates after cleaning.")

    chosen_description = pick_best_description(cleaned_desc_candidates, subject)

    # ── Tags: refine & enrich ─────────────────────────────────
    tags = meta.get("tags", [])
    clean_tags = []
    seen = set()

    for t in tags:
        t_clean = str(t).strip()
        if not t_clean:
            continue
        if t_clean.startswith("#"):
            t_clean = t_clean[1:]
        t_clean = re.sub(r"\s+", " ", t_clean).strip()
        t_lower = t_clean.lower()
        if t_lower not in seen:
            seen.add(t_lower)
            clean_tags.append(t_clean)

    # Enrich with subject-based SEO tags
    subject_keywords = extract_subject_keywords(subject)
    base_subject_tags = [
        subject,
        f"{subject} explained",
        f"{subject} for beginners",
        f"{subject} made simple",
        f"{subject} song",
        f"{subject} tutorial",
    ]
    for kw in subject_keywords:
        base_subject_tags.append(f"{kw} basics")
        base_subject_tags.append(f"what is {kw}")
    for t in base_subject_tags:
        t_clean = t.strip()
        if not t_clean:
            continue
        t_lower = t_clean.lower()
        if t_lower not in seen:
            seen.add(t_lower)
            clean_tags.append(t_clean)

    # Limit total number of tags to something reasonable (upload step further sanitizes)
    clean_tags = clean_tags[:40]

    # ── Pinned comment ────────────────────────────────────────
    pinned_comment = meta.get("pinned_comment", "") or ""
    if not pinned_comment.strip():
        pinned_comment = (
            "What topic should I turn into a song next? "
            "Drop your ideas in the comments!"
        )

    # Build final metadata dict
    final_meta = {
        "title": final_title,
        "title_candidates": cleaned_title_candidates,
        "description": chosen_description,
        "description_candidates": cleaned_desc_candidates,
        "tags": clean_tags,
        "pinned_comment": pinned_comment,
    }

    return final_meta


# ── Generate & save metadata ─────────────────────────────────

print("🧾 Generating YouTube metadata with GPT (A/B titles & descriptions)...")
video_meta = generate_video_metadata(
    subject=subject,
    persona=persona,
    tone=tone,
    lyrics=lyrics_text,
    duration_sec=duration_sec,
    language=LYRICS_LANGUAGE,
)

METADATA_JSON_PATH.write_text(json.dumps(video_meta, indent=2, ensure_ascii=False), encoding="utf-8")

print("✅ Metadata generated and saved to:", METADATA_JSON_PATH)
print("\n──── FINAL CHOSEN TITLE ────")
print(video_meta["title"])
print("\n──── TITLE CANDIDATES ────")
for i, t in enumerate(video_meta.get("title_candidates", []), start=1):
    print(f"{i}. {t}")
print("\n──── DESCRIPTION (preview) ────")
print(video_meta["description"][:800], "...\n")
print("\n──── TAGS (first 20) ────")
print(video_meta["tags"][:20])
print("\n──── PINNED COMMENT SUGGESTION ────")
print(video_meta["pinned_comment"])


In [ ]:
# ─────────────────────────────────────────────────────────────
# 10. Upload final video to YouTube (with robust tag fallback)
# ─────────────────────────────────────────────────────────────

import json
import time
import re
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError

# We reuse:
# - config (for youtube_privacy)
# - get_authenticated_service (from Block 2)
# - RUN_DIR
# - FINAL_MP4 (from Block 8)
# - METADATA_JSON_PATH (from Block 9)
# - subject (from Block 3)

# Sanity checks
if not FINAL_MP4.exists():
    raise FileNotFoundError(f"Final video not found at {FINAL_MP4}. Run Block 8 first.")

if not METADATA_JSON_PATH.exists():
    raise FileNotFoundError(f"Metadata JSON not found at {METADATA_JSON_PATH}. Run Block 9 first.")

video_meta = json.loads(METADATA_JSON_PATH.read_text(encoding="utf-8"))

title = video_meta["title"]
description = video_meta["description"]
tags = video_meta.get("tags", [])
pinned_comment_suggestion = video_meta.get("pinned_comment", "")

# Ensure tags is a list of strings
if isinstance(tags, str):
    tags = [t.strip() for t in tags.split(",") if t.strip()]


def get_youtube_client_safe():
    """
    Reuse global youtube_client if it exists, otherwise create a new one.
    """
    try:
        yt = youtube_client  # from preflight, if still in memory
        _ = yt.videos  # simple sanity check
        print("🔁 Reusing existing YouTube client.")
        return yt
    except Exception:
        print("🔄 Creating new YouTube client via get_authenticated_service()...")
        return get_authenticated_service()


def clean_youtube_tags(raw_tags):
    """
    Strong cleaning so YouTube is less likely to reject tags.

    Strategy:
    - Remove '#'
    - Keep only letters, digits, spaces, and a small safe set of punctuation: - & ' ! ? .
    - Truncate each tag to 30 characters
    - Deduplicate (case-insensitive)
    - Trim list so total length <= 450 chars
    """
    allowed_pattern = re.compile(r"[^a-zA-Z0-9 \-\&\'!\?\.]", re.UNICODE)

    cleaned = []
    seen = set()
    total_len = 0
    MAX_TAG_LEN = 30
    MAX_TOTAL_LEN = 450

    for t in raw_tags:
        t = str(t).strip()
        if not t:
            continue

        # Remove leading '#'
        if t.startswith("#"):
            t = t[1:].strip()
        if not t:
            continue

        # Remove disallowed chars
        t = allowed_pattern.sub("", t)

        # Collapse multiple spaces
        t = re.sub(r"\s+", " ", t).strip()

        if not t:
            continue

        # Truncate
        if len(t) > MAX_TAG_LEN:
            t = t[:MAX_TAG_LEN].rstrip()

        t_lower = t.lower()
        if t_lower in seen:
            continue

        # Check total length budget
        if total_len + len(t) > MAX_TOTAL_LEN:
            break

        cleaned.append(t)
        seen.add(t_lower)
        total_len += len(t)

    return cleaned


def upload_video_to_youtube_attempt(
    youtube,
    file_path,
    title,
    description,
    tags_attempt,
    category_id="27",  # 'Education'
    privacy="public",
    chunk_size=1024 * 1024,
    max_retries=5,
    label="",
):
    """
    Single upload attempt with a specific tag set.
    Raises HttpError if YouTube rejects tags or other non-transient errors.
    """
    # Ensure #shorts in description for Shorts distribution
    if "#shorts" not in description.lower():
        description_to_send = description.rstrip() + "\n\n#shorts"
    else:
        description_to_send = description

    print(f"\n🔖 Attempt '{label}': sending tags:")
    print(tags_attempt)
    print("Total chars in tags:", sum(len(t) for t in tags_attempt))

    snippet = {
        "title": title,
        "description": description_to_send,
        "categoryId": str(category_id),
    }

    # Only include tags field if we actually have any
    if tags_attempt:
        snippet["tags"] = tags_attempt

    body = {
        "snippet": snippet,
        "status": {
            "privacyStatus": privacy,
            "selfDeclaredMadeForKids": False,
        },
    }

    media = MediaFileUpload(
        str(file_path),
        mimetype="video/mp4",
        chunksize=chunk_size,
        resumable=True,
    )

    request = youtube.videos().insert(
        part="snippet,status",
        body=body,
        media_body=media,
    )

    print(f"🚀 Starting YouTube upload (attempt '{label}')...")
    response = None
    retry = 0

    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"🟢 Upload progress: {int(status.progress() * 100)}%")
        except HttpError as e:
            # invalidTags error is handled by the caller (for fallback)
            if e.resp.status == 400 and "invalidTags" in str(e):
                print(f"❌ YouTube reports invalid tags on attempt '{label}'.")
                raise

            # Transient server errors → exponential backoff
            if e.resp.status in [500, 502, 503, 504] and retry < max_retries:
                retry += 1
                wait_for = 2 ** retry
                print(f"⚠️ Transient YouTube error ({e.resp.status}). Retry {retry}/{max_retries} in {wait_for}s...")
                time.sleep(wait_for)
                continue
            else:
                print(f"❌ Non-recoverable YouTube error on attempt '{label}': {e}")
                raise
        except Exception as e:
            if retry < max_retries:
                retry += 1
                wait_for = 2 ** retry
                print(f"⚠️ Unexpected error during upload (attempt '{label}'): {e}. "
                      f"Retry {retry}/{max_retries} in {wait_for}s...")
                time.sleep(wait_for)
                continue
            else:
                print(f"❌ Max retries reached on attempt '{label}'. Upload failed.")
                raise

    if "id" not in response:
        raise RuntimeError(f"Unexpected YouTube API response on attempt '{label}': {response}")

    video_id = response["id"]
    print(f"✅ Upload complete on attempt '{label}'! Video ID:", video_id)
    return video_id


def upload_video_to_youtube_with_fallback(
    youtube,
    file_path,
    title,
    description,
    tags,
    category_id="27",
    privacy="public",
):
    """
    Try uploading with:
    1) Full cleaned tags
    2) If invalidTags: small conservative tag set
    3) If still invalidTags: no tags at all

    This guarantees that tags can never break the entire run.
    """
    # 1) Full cleaned tags
    full_cleaned = clean_youtube_tags(tags)

    try:
        return upload_video_to_youtube_attempt(
            youtube=youtube,
            file_path=file_path,
            title=title,
            description=description,
            tags_attempt=full_cleaned,
            category_id=category_id,
            privacy=privacy,
            label="full-cleaned-tags",
        )
    except HttpError as e:
        if e.resp.status == 400 and "invalidTags" in str(e):
            print("⚠️ YouTube rejected the full tag set (invalidTags). Falling back to conservative tags...")
        else:
            # Some other error; re-raise
            raise

    # 2) Conservative tag set, very safe
    conservative_raw = [
        subject,
        f"{subject} explained",
        "educational song",
        "learning with music",
        "explained simply",
    ]
    conservative_cleaned = clean_youtube_tags(conservative_raw)

    try:
        return upload_video_to_youtube_attempt(
            youtube=youtube,
            file_path=file_path,
            title=title,
            description=description,
            tags_attempt=conservative_cleaned,
            category_id=category_id,
            privacy=privacy,
            label="conservative-tags",
        )
    except HttpError as e:
        if e.resp.status == 400 and "invalidTags" in str(e):
            print("⚠️ YouTube rejected even conservative tags (invalidTags). "
                  "Final fallback: uploading with NO tags.")
        else:
            raise

    # 3) Final fallback: no tags at all
    return upload_video_to_youtube_attempt(
        youtube=youtube,
        file_path=file_path,
        title=title,
        description=description,
        tags_attempt=[],  # no tags
        category_id=category_id,
        privacy=privacy,
        label="no-tags",
    )


# ── Run the upload ───────────────────────────────────────────

youtube = get_youtube_client_safe()
privacy = config.youtube_privacy  # from Block 1 config

video_id = upload_video_to_youtube_with_fallback(
    youtube=youtube,
    file_path=FINAL_MP4,
    title=title,
    description=description,
    tags=tags,
    category_id="27",  # Education
    privacy=privacy,
)

video_url = f"https://youtu.be/{video_id}"
print("\n🎉 Video is live (or queued) on YouTube:")
print("   ", video_url)

# Show pinned comment suggestion again for easy copy-paste
if pinned_comment_suggestion:
    pinned_path = RUN_DIR / "logs" / "pinned_comment.txt"
    pinned_path.write_text(pinned_comment_suggestion, encoding="utf-8")
    print("\n📌 Pinned comment suggestion (also saved to pinned_comment.txt):")
    print(pinned_comment_suggestion)
    print("\n   File:", pinned_path)
